# Imports

In [ ]:
# Our tools
import exoplanet as xo
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import lightkurve as lk
from astropy.io import fits
from astropy.timeseries import BoxLeastSquares
from scipy import stats
import graphviz
import astropy.units as u
from astropy.timeseries import LombScargle

import pymc3 as pm
import aesara_theano_fallback.tensor as tt
import pymc3_ext as pmx
from functools import partial
from scipy import constants

import os
import warnings
#os.environ["MKL_THREADING_LAYER"] = "GNU"
#warnings.filterwarnings("ignore", category=DeprecationWarning)
#warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
plt.rcParams.update(plt.rcParamsDefault)
%matplotlib inline
plt.style.use('seaborn-dark-palette')

In [ ]:
plt.rcParams["figure.figsize"]=8,6
plt.rcParams.update({'font.size': 16})

# Data 

## Tess

In [ ]:
#sectors times, 41-48
from astropy.time import Time
sectors = ['2019-07-18T20:30:00', '2019-08-14T17:05:00', '2020-01-21T22:25:00', '2020-02-18T06:55:00', '2021-07-24T11:45:00','2021-08-20T02:05:00','2022-01-28T10:25:00','2022-02-25T11:50:00']
sectors_times = Time(sectors, format='fits')
sectors_times_btjd = sectors_times.jd-2457000
sectors_times_btjd

In [ ]:
lk.search_targetpixelfile('TIC 99869022')

In [ ]:
tpf = lk.search_targetpixelfile('TIC 99869022', exptime='short', sector=41).download_all(quality_bitmask='hard')

In [ ]:
tpf.plot()

In [ ]:
# !python ~/Descargas/tpfplotter/tpfplotter.py 99869022 --maglim 6 --sector 21

In [ ]:
lk.search_lightcurve('TIC 99869022')

In [ ]:
lk.search_lightcurve('TIC 99869022', exptime=1800)

In [ ]:
lc_file_long = lk.search_lightcurve('TIC 99869022', exptime=1800, author='TESS-SPOC').download_all(flux_column="pdcsap_flux")

In [ ]:
lc_file_long.plot()

In [ ]:
lc_file = lk.search_lightcurve('TIC 99869022', author='SPOC').download_all(flux_column="pdcsap_flux")
lc = lc_file.stitch().remove_nans().normalize().remove_outliers()
time = lc.time.value
flux = lc.flux

lc_long = lc_file_long.stitch().remove_nans().normalize()
time_long = lc_long.time.value
flux_long = lc_long.flux

with fits.open(lc_file[0].filename) as hdu:
    hdr = hdu[1].header
with fits.open(lc_file_long[0].filename) as hdu:
    hdr_long = hdu[1].header

texp_long = hdr_long["FRAMETIM"] * hdr_long["NUM_FRM"]
texp_long /= 60.0 * 60.0 * 24.0
texp = hdr["FRAMETIM"] * hdr["NUM_FRM"]
texp /= 60.0 * 60.0 * 24.0

ref_time = 0.5 * (np.min(time) + np.max(time))
x_ = np.ascontiguousarray(time - ref_time, dtype=np.float64)
y_ = np.ascontiguousarray(1e3 * (flux - 1.0), dtype=np.float64) # Here we convert flux to ppt
yerr_ = np.ascontiguousarray(1e3 * lc.flux_err, dtype=np.float64) 

xlong_ = np.ascontiguousarray(time_long - ref_time, dtype=np.float64)
ylong_ = np.ascontiguousarray(1e3 * (flux_long - 1.0), dtype=np.float64) # Here we convert flux to ppt
yerrlong_ = np.ascontiguousarray(1e3 * lc_long.flux_err, dtype=np.float64)

plt.figure(figsize=(12, 6))
plt.plot(time, y_, "o", color='black',markersize=2)
plt.plot(time_long, ylong_, "o", color='red',markersize=2)
plt.xlabel("Time [BTJD]")
plt.ylabel("Relative flux")
#_ = plt.xlim(x_.min(), x_.max())
#plt.savefig('tessdata_1199.png',dpi=300,bbox_inches='tight')

In [ ]:
fig, (ax1, ax2, ax3, ax4) = plt.subplots(1,4, sharey=True, figsize=(16,4), gridspec_kw={'width_ratios': [1,1,1,1]})
fig.subplots_adjust(wspace=0.05)
ax1.plot(time_long, ylong_, "o", color='#348ABD', markersize=1, alpha=0.6)
ax2.plot(time_long, ylong_, "o", color='#348ABD', markersize=1, alpha=0.6)
ax3.plot(time, y_, "o", color='#348ABD', markersize=1, alpha=0.6)
ax4.plot(time, y_, "o", color='#348ABD', markersize=1, alpha=0.6)
off=5
ax1.set_xlim(sectors_times_btjd[0]-off, sectors_times_btjd[1]+off)  
ax2.set_xlim(sectors_times_btjd[2]-off, sectors_times_btjd[3]+off)  
ax3.set_xlim(sectors_times_btjd[4]-off, sectors_times_btjd[5]+off)
ax4.set_xlim(sectors_times_btjd[6]-off, sectors_times_btjd[7]+off)
#ax2.set_xticks([2600, 2650, 2700])
ax1.spines.right.set_visible(False)
ax2.spines.left.set_visible(False)
ax2.spines.right.set_visible(False)
ax3.spines.left.set_visible(False)
ax3.spines.right.set_visible(False)
ax4.spines.left.set_visible(False)

ax1.yaxis.tick_left()
ax2.yaxis.tick_right()
ax3.yaxis.tick_right()
ax4.yaxis.tick_right()
ax1.tick_params(labelright=False)
ax2.tick_params(axis='y', which='both', left=False, right=False, 
                labelright=False, labelleft=False)
ax3.tick_params(axis='y', which='both', left=False, right=False, 
                labelright=False, labelleft=False)
ax4.tick_params(labelleft=False)
ax1.tick_params(axis='both', which='major', labelsize=13)
ax2.tick_params(axis='both', which='major', labelsize=13)
ax3.tick_params(axis='both', which='major', labelsize=13)
ax4.tick_params(axis='both', which='major', labelsize=13)
d = .5  
kwargs = dict(marker=[(-1, -d), (1, d)], markersize=12,
              linestyle="none", color='k', mec='k', mew=1, clip_on=False)
ax1.plot([1, 1], [0, 1], transform=ax1.transAxes, **kwargs)
ax2.plot([0, 0], [1, 0], transform=ax2.transAxes, **kwargs)
ax2.plot([1, 1], [0, 1], transform=ax2.transAxes, **kwargs)
ax3.plot([0, 0], [1, 0], transform=ax3.transAxes, **kwargs)
ax3.plot([1, 1], [0, 1], transform=ax3.transAxes, **kwargs)
ax4.plot([0, 0], [1, 0], transform=ax4.transAxes, **kwargs)
for i in sectors_times_btjd[:2]:
    ax1.axvline(x=i, color='gray', linestyle='--', linewidth=1.3, alpha=0.8)
for i in sectors_times_btjd[2:4]:
    ax2.axvline(x=i, color='gray', linestyle='--', linewidth=1.3, alpha=0.8)
for i in sectors_times_btjd[4:6]:
    ax3.axvline(x=i, color='gray', linestyle='--', linewidth=1.3, alpha=0.8)
for i in sectors_times_btjd[6:]:
    ax4.axvline(x=i, color='gray', linestyle='--', linewidth=1.3, alpha=0.8)
fig.text(0.5, 0.001, "Time (BJD - 2,457,000)", ha='center', va='center', size=14)
fig.text(0.095, 0.5, u'$\Delta Flux$ (‰)', ha='center', va='center', rotation='vertical', size=14)
ax1.text(sectors_times_btjd[0]+(sectors_times_btjd[1]-sectors_times_btjd[0])/2, 5.33, 
    "S14", ha="center", va="center", size=12, fontfamily='serif',
    bbox=dict(boxstyle="circle,pad=0.15", fc='white', lw=0.5))
ax2.text(sectors_times_btjd[2]+(sectors_times_btjd[3]-sectors_times_btjd[2])/2, 5.33, 
    "S21", ha="center", va="center", size=12, fontfamily='serif',
    bbox=dict(boxstyle="circle, pad=0.15", fc='white', lw=0.5))
ax3.text(sectors_times_btjd[4]+(sectors_times_btjd[5]-sectors_times_btjd[4])/2, 5.33, 
    "S41", ha="center", va="center", size=12, fontfamily='serif',
    bbox=dict(boxstyle="circle, pad=0.15", fc='white', lw=0.5))
ax4.text(sectors_times_btjd[6]+(sectors_times_btjd[7]-sectors_times_btjd[6])/2, 5.33, 
    "S48", ha="center", va="center", size=12, fontfamily='serif',
    bbox=dict(boxstyle="circle, pad=0.15", fc='white', lw=0.5))
#plt.savefig('tessdata_1199.png',dpi=300,bbox_inches='tight', facecolor='white')
plt.show()

BLS

In [ ]:
xt = np.concatenate((xlong_, x_), axis=0)
yt = np.concatenate((ylong_, y_), axis=0)
yerrt = np.concatenate((yerrlong_, yerr_), axis=0)

In [ ]:
period_grid = np.exp(np.linspace(np.log(1), np.log(15), 30000)) #1 y 15 -- 1 y 30
durations = np.exp(np.linspace(np.log(0.01), np.log(0.2), 100))

bls = BoxLeastSquares(x_, y_, yerr_)
bls_power = bls.power(period_grid, durations, oversample=30)

# Save the highest peak as the planet candidate
index = np.argmax(bls_power.power)
bls_period = bls_power.period[index]
bls_t0 = bls_power.transit_time[index]
bls_depth = bls_power.depth[index]
bls_duration = bls_power.duration[index]

print('bls period:', bls_period)
print('bls t0:', bls_t0)
print('bls depth:', bls_depth)
print('ref_time:', ref_time)
print('epoch:', bls_t0 + ref_time)
print('bls duration:', bls_duration*24)

In [ ]:
# tess project fit
spoc_depth = 4.340 
spoc_duration = 2.149/24
spoc_period = 3.671463

In [ ]:
# 50000, 1000, 30
# bls period: 3.6714776169735264
# bls t0: -107.44944712249175
# bls depth: 3.736413756919435
# ref_time: 2527.9875009679236
# epoch: 2420.538053845432
# bls duration: 1.9840000000000007

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(9, 6), gridspec_kw={'height_ratios': [1, 2]})
fig.subplots_adjust(hspace=0.35)

ax1.axvline(np.log10(bls_period), color="red", lw=5, alpha=0.4)
ax1.axvline(np.log10(bls_period/2), linestyle='dotted', color="red", lw=4, alpha=0.4)
ax1.axvline(np.log10(bls_period/3), linestyle='dotted', color="red", lw=4, alpha=0.4)
#ax1.axvline(np.log10(bls_period/4), linestyle='dotted', color="red", lw=4, alpha=0.4)
ax1.axvline(np.log10(bls_period*2), linestyle='dotted', color="red", lw=4, alpha=0.4)
ax1.axvline(np.log10(bls_period*3), linestyle='dotted', color="red", lw=4, alpha=0.4)
ax1.axvline(np.log10(bls_period*4), linestyle='dotted', color="red", lw=4, alpha=0.4)
ax1.plot(np.log10(bls_power.period), bls_power.power, "k")
ax1.set_ylim(0,3500)
ax1.set_ylabel("Power", labelpad=32)
ax1.set_yticks([])
ax1.set_xlim(np.log10(period_grid.min())+0.001, np.log10(period_grid.max()))
ax1.set_xlabel(r"$\log_{10}~ P$ [d]")

# Plot the folded transit
x_fold_ = (x_ - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
x_fold_long = (xlong_ - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
xt_fold = (xt - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
# x_fold_ = (x_ - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# x_fold_long = (xlong_ - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# xt_fold = (xt - t0_ + 0.5 * p_) % p_ - 0.5 * p_

# ax2.plot(x_fold_long*24, ylong_, ".", alpha=0.6, ms=4, color='r', zorder=102)
# ax2.errorbar(x_fold_long*24, ylong_, yerr=yerrlong_, color='r', fmt="none", capsize=0, elinewidth=1, zorder=10000, alpha=0.5)
# ax2.plot(x_fold_*24, y_, ".", alpha=0.7, ms=5, color='#348ABD', zorder=101)
# ax2.errorbar(x_fold_*24, y_, yerr=yerr_, fmt="none", color='#348ABD', elinewidth=1, alpha=0.2, capsize=0, zorder=10000)
ax2.plot(xt_fold*24, yt, ".", alpha=0.4, ms=7, color='#348ABD', zorder=100)
ax2.errorbar(xt_fold*24, yt, yerr=yerrt, fmt="none", color='#348ABD', elinewidth=1, alpha=0.4, capsize=0)
ax2.set_ylim(-8,6)
ax2.set_yticks([-8,-6,-4,-2,0,2,4,6])

#Overplot the phase binned light curve
lcc = lk.LightCurve(time=xt_fold, flux=yt, flux_err=yerrt)
lcc_binned = lcc.bin(time_bin_size=0.01)
ax2.errorbar(lcc_binned['time'].value*24, lcc_binned['flux'].value, yerr=lcc_binned['flux_err'].value, fmt='o', color='k', 
                ms=5, markeredgecolor='k', markerfacecolor='k', markeredgewidth=0, capsize=2, alpha=0.8)
ax2.set_xlim(-0.11*24, 0.11*24)
ax2.set_ylabel(u'$\Delta Flux$ [‰]', labelpad=0)
_ = ax2.set_xlabel("Time since transit [h]")

# plot the BLS model
x_bin = np.linspace(-0.3, 0.3, 1000)
f = bls.model(x_bin + bls_t0, bls_period, bls_duration, bls_t0)
ax2.plot(x_bin*24, f, lw=1.75)
plt.tight_layout()

#plt.savefig('bls_paper_1199.png', dpi=300, bbox_inches='tight', facecolor='white')

In [ ]:
transit_mask = bls.transit_mask(x_, bls_period, 0.2, bls_t0)
transit_mask_long = bls.transit_mask(xlong_, bls_period, 0.2, bls_t0)
x_fold_ = (x_ - bls_t0 + 0.5*bls_period) % bls_period - 0.5*bls_period

# Me quedo solo con los puntos cerca de los transitos
x_fold = x_fold_[transit_mask]
x = x_[transit_mask]
y = y_[transit_mask]
yerr = yerr_[transit_mask]

xlong = xlong_[transit_mask_long]
ylong = ylong_[transit_mask_long]
yerrlong = yerrlong_[transit_mask_long]

plt.scatter(x_fold, y, c='k', s=2)

plt.xlabel("time since transit [days]")
plt.ylabel("relative flux [ppt]")
#plt.xlim(-0.12,0.12)

In [ ]:
len(y)

## SOPHIE RVs

In [ ]:
data = pd.read_table('./data/rvs/1199_final_rvs.dat', sep='\s+')
data.rename(columns={'rv(km/s)': 'rv', 'sigRV(km/s)':'err', 'bis(km/s)': 'bis'}, inplace=True)
snr= pd.read_table('./data/rvs/SNR_1199', sep='\s+', header=None) # SNR

x_rv = np.array(data.bjd+2400000-2457000)-ref_time
y_rv = np.array((data.rv-data.rv.mean())*1000)
yerr_rv = np.array(data.err*1000)
fig, ax = plt.subplots(figsize=(7,4))
# horizontal line at 0
ax.axhline(y=0, color='k', linestyle='--', linewidth=0.8, alpha=0.8)
ax.errorbar(data.bjd+2400000-2457000, y_rv, yerr=yerr_rv, fmt=".k", markersize=8, capsize=0, elinewidth=0.5, alpha=0.8)
ax.set_xlabel("Time [TBJD]", fontsize=12)
ax.set_ylabel("$\Delta_{RV}$ (m/s)", fontsize=12)
ax.set_title('TOI-1199 b', fontsize=13)
ax.set_ylim(-70,70)
ax.tick_params(axis='both', which='major', labelsize=13)
plt.savefig('rvs_1199.png',dpi=300,bbox_inches='tight', facecolor='white')


In [ ]:
snr[2].values.mean()

In [ ]:
# Compute a reference time that will be used to normalize the trends model
x_ref = 0.5 * (x_rv.min() + x_rv.max())

K = xo.estimate_semi_amplitude(bls_period, x_rv, y_rv, yerr_rv, t0s=bls_t0)
print(K, "m/s")

# Stellar parameters from Sousa
M_star = 1.23, 0.07
R_star = 1.45, 0.03
teff = 5710

msini = xo.estimate_minimum_mass(bls_period, x_rv, y_rv, yerr_rv, t0s=bls_t0, m_star=M_star[0])
msini = msini.to(u.M_earth)
print('minimum mass=', msini)

Periodograms

In [ ]:
# read de BIS data
ls = LombScargle(data.bjd, data.rv, data.err)
ls_bis = LombScargle(data.bjd, data.bis)
ls_w = LombScargle(data.bjd, np.ones(len(data.bjd)), data.err, fit_mean=False, center_data=False)

frequency, power = ls.autopower(minimum_frequency=0.001, maximum_frequency=2, samples_per_peak=5)

frequency_bis, power_bis = ls_bis.autopower(minimum_frequency=0.001, maximum_frequency=2, samples_per_peak=5)

frequency_window, power_window = ls_w.autopower(minimum_frequency=0.001, maximum_frequency=2, samples_per_peak=15)

probabilities = [0.1, 0.05, 0.01]
faps = ls.false_alarm_level(probabilities)  
faps_bis = ls_bis.false_alarm_level(probabilities)


In [ ]:
# aliases computed in DACE
aliases = [1.374, 1.369, 0.786, 0.784]

In [ ]:
from matplotlib.ticker import ScalarFormatter
# plot two figures in a column
fig, axes = plt.subplots(3, 1, sharex=False, figsize=(10, 6))
axes[0].plot(1/frequency, power, label='RVs') 
axes[1].plot(1/frequency_bis, power_bis, label='BIS') 
axes[2].plot(1/frequency_window, power_window, label='Window function')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)

axes[0].set_xlabel('')
axes[0].set_xlim(0.6, 1000)
axes[0].set_ylim(0, 1.0)
axes[1].set_xlabel('')
axes[1].set_xlim(0.6, 1000)
axes[1].set_ylim(0, 1.0)
axes[2].set_xlabel('Period [d]')
axes[2].set_xlim(0.6, 1000)
axes[2].set_ylim(0, 1.0)

axes[0].set_xscale('log')
axes[1].set_xscale('log')
axes[2].set_xscale('log')

axes[0].set_ylabel('power')
axes[1].set_ylabel('power')
axes[2].set_ylabel('power')
axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')
axes[2].legend(loc='upper right')
# overplot the false alarm probabilities
for i, prob in enumerate(probabilities):
    axes[0].axhline(faps[i], ls=':', color='black', label='{}%'.format(prob))
    axes[1].axhline(faps_bis[i], ls=':', color='black', label='{}%'.format(prob))

# plot the 'bls_period' as a vertical line behind the plot
axes[0].axvline(bls_period, ls=':', color='red', label='bls period')
axes[1].axvline(bls_period, ls=':', color='red', label='bls period')

formatter = ScalarFormatter()
axes[0].xaxis.set_major_formatter(formatter)
axes[1].xaxis.set_major_formatter(formatter)
#plt.savefig('bisrv_periodogram_1199.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
def windowfunction(t, erv, prange):

    w = (1/erv**2)/np.sum(1/erv**2)

    wf = np.zeros(len(prange))

    for i in range(len(prange)):

        omega = 2 * np.pi / prange[i]

        ccos = np.cos(omega*t)
        ssin = np.sin(omega*t)

        wC = np.sum(w*ccos)
        wS = np.sum(w*ssin)

        wC = np.sum(ccos)/len(ccos)
        wS = np.sum(ssin)/len(ssin)

        wf[i] = wC * wC + wS * wS
    return wf

In [ ]:
prange = np.logspace(-1, 3, 100000)
wf = windowfunction(data.bjd, data.err, prange)
wf

In [ ]:
fig, axes = plt.subplots(3, 1, sharex=False, figsize=(10, 6))
axes[0].plot(1/frequency, power, label='RVs') 
axes[1].plot(1/frequency_bis, power_bis, label='BIS') 
axes[2].plot(prange, wf, label='Window function')
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.3)

axes[0].set_xlabel('')
axes[0].set_xlim(0.6, 1000)
axes[0].set_ylim(0, 1.0)
axes[1].set_xlabel('')
axes[1].set_xlim(0.6, 1000)
axes[1].set_ylim(0, 1.0)
axes[2].set_xlabel('Period [d]')
axes[2].set_xlim(0.6, 1000)
axes[2].set_ylim(0, 1.0)

axes[0].set_xscale('log')
axes[1].set_xscale('log')
axes[2].set_xscale('log')

axes[0].set_ylabel('power')
axes[1].set_ylabel('power')
axes[2].set_ylabel('power')
axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')
axes[2].legend(loc='upper right')
# overplot the false alarm probabilities
for i, prob in enumerate(probabilities):
    axes[0].axhline(faps[i], ls=':', color='black', label='{}%'.format(prob))
    axes[1].axhline(faps_bis[i], ls=':', color='black', label='{}%'.format(prob))

# plot the 'bls_period' as a vertical line behind the plot
axes[0].axvline(bls_period, ls=':', color='red', label='bls period')
axes[1].axvline(bls_period, ls=':', color='red', label='bls period')

formatter = ScalarFormatter()
axes[0].xaxis.set_major_formatter(formatter)
axes[1].xaxis.set_major_formatter(formatter)
#plt.savefig('bisrv_periodogram_1199.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
fig, axes = plt.subplots(2,1, figsize=(10, 6))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0.2)
axes[0].plot(1/frequency_window, power_window, label='Window function (LS)')
axes[1].plot(prange, wf, label='Window function (Rodrigo)')
axes[0].set_xlabel('')
axes[1].set_xlabel('Period [d]')
axes[0].set_xlim(0.6, 1000)
axes[0].set_ylim(0, 1.0)
axes[1].set_xlim(0.6, 1000)
axes[1].set_ylim(0, 1.0)
axes[0].set_xscale('log')
axes[1].set_xscale('log')
axes[0].set_ylabel('power')
axes[1].set_ylabel('power')
axes[0].legend(loc='upper right')
axes[1].legend(loc='upper right')
#plt.savefig('windowfunction_1199.png', dpi=300, bbox_inches='tight', facecolor='white')

BIS vs rvs

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6, 5))
g = ax.scatter(y_rv, data.bis*1e3, c=data.bjd, cmap='viridis_r', s=50, zorder=10)
# add errorbars on data.rv
ax.errorbar(y_rv, data.bis*1e3, xerr=yerr_rv, fmt='none', ecolor='k', markersize=8, zorder=-10)
ax.set_xlabel("$\Delta$RV [m/s]")
ax.set_ylabel("BIS [m/s]")
fig.colorbar(g, label='BJD - 2,400,000 (days)')
#plt.savefig('bisrv_1199.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
0.5*np.log(bls_depth * 1e-3)

In [ ]:
np.log(np.median(yerr_rv))

## Ground LCs

### KeplerCam

In [ ]:
# filtros griz sloan: B y z'
# B     440nm		483.5nm	                        43.5nm
# z'	>820nm	    966.5nm	                        255.8nm	

In [ ]:
data1 = pd.read_table('./data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0213_KeplerCam_B.dat', sep='\s+')
data2 = pd.read_table('./data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0413_KeplerCam_B.dat', sep='\s+')
data3 = pd.read_table('./data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0213_KeplerCam_z.dat', sep='\s+')
data4 = pd.read_table('./data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0413_KeplerCam_z.dat', sep='\s+')
data1.columns

In [ ]:
#puntos?
print('puntos:', len(data1)+len(data2)+len(data3)+len(data4))
# texp?
print('texp:', np.median(np.diff(data1['BJD_TDB']))*24*60*60)
texp_keplercam = None

In [ ]:
data1['flux'] = ((data1['rel_flux_T1']/(data1['rel_flux_T1'].median())))*1e3
data2['flux'] = ((data2['rel_flux_T1']/(data2['rel_flux_T1'].median())))*1e3
data3['flux'] = ((data3['rel_flux_T1']/(data3['rel_flux_T1'].median())))*1e3
data4['flux'] = ((data4['rel_flux_T1']/(data4['rel_flux_T1'].median())))*1e3

data1['flux_err'] = ((data1['rel_flux_err_T1']/(data1['rel_flux_T1'].median())))*1e3
data1['flux_err'] = ((data1['rel_flux_err_T1']/(data1['rel_flux_T1'].median())))*1e3
data2['flux_err'] = ((data2['rel_flux_err_T1']/(data2['rel_flux_T1'].median())))*1e3
data3['flux_err'] = ((data3['rel_flux_err_T1']/(data3['rel_flux_T1'].median())))*1e3
data4['flux_err'] = ((data4['rel_flux_err_T1']/(data4['rel_flux_T1'].median())))*1e3

data1['time'] = data1['BJD_TDB']-2457000-ref_time
data2['time'] = data2['BJD_TDB']-2457000-ref_time
data3['time'] = data3['BJD_TDB']-2457000-ref_time
data4['time'] = data4['BJD_TDB']-2457000-ref_time

In [ ]:
out_mask3 = np.abs(data3.flux-data4.flux.mean())<3*data3.flux.std()
out_mask1 = np.abs(data1.flux-data2.flux.mean())<3*data1.flux.std()
data3 = data3[out_mask3]
data1 = data1[out_mask1]

k_mask1 = bls.transit_mask(data1['time'], bls_period, 0.2, bls_t0)
k_mask2 = bls.transit_mask(data2['time'], bls_period, 0.2, bls_t0)
k_mask3 = bls.transit_mask(data3['time'], bls_period, 0.2, bls_t0)
k_mask4 = bls.transit_mask(data4['time'], bls_period, 0.2, bls_t0)

data1 = data1[k_mask1]
data2 = data2[k_mask2]
data3 = data3[k_mask3]
data4 = data4[k_mask4]

In [ ]:
#puntos?
print('puntos:', len(data1)+len(data2)+len(data3)+len(data4))

In [ ]:
# Lest plot them together
plt.plot(data1['time'], data1['flux'], 'o', label='2021.0213_KeplerCam_B')
plt.plot(data2['time'], data2['flux'], 'o', label='2021.0413_KeplerCam_B')
plt.plot(data3['time'], data3['flux'], 'o', label='2021.0213_KeplerCam_z')
plt.plot(data4['time'], data4['flux'], 'o', label='2021.0413_KeplerCam_z')
plt.xlim(-211, -210)
#plt.xlim(-269.2,-268.5)
plt.show()

In [ ]:
# Plot the folded data, con los parametros del BLS
data1_fold = (data1['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data2_fold = (data2['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data3_fold = (data3['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data4_fold = (data4['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period

plt.figure(figsize=(10, 5))
#plt.plot(data1_fold*24, data1['flux'], ".k", label="data1", zorder=-1000)
#plt.plot(data2_fold*24, data2['flux'], ".k", label="data2", zorder=-1000)
plt.plot(data3_fold*24, data3['flux'], ".k", label="data3", zorder=-1000)
#plt.plot(data4_fold*24, data4['flux'], ".k", label="data4", zorder=-1000)

plt.ylabel("de-trended flux (ppt)")
plt.xlabel('time since transit (hours)')
plt.xlim(-0.10*24, 0.10*24)
plt.ylim(990,1010)
plt.title("TOI1199")
#plt.savefig('new_phot_1199.png',dpi=200,bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# ajustemos un pol grado 2 a los datos oot
# mascaras para sacar los datos en transito
oot_mask1 = bls.transit_mask(data1['time'], spoc_period, spoc_duration, bls_t0)
oot_mask2 = bls.transit_mask(data2['time'], spoc_period, spoc_duration, bls_t0)
oot_mask3 = bls.transit_mask(data3['time'], spoc_period, spoc_duration, bls_t0)
oot_mask4 = bls.transit_mask(data4['time'], spoc_period, spoc_duration, bls_t0)

In [ ]:
# checking the oot mask
plt.figure(figsize=(10, 5))
plt.plot((data1_fold*24)[~oot_mask1], data1['flux'][~oot_mask1], ".k", label="data1", zorder=-1000)
plt.plot((data2_fold*24)[~oot_mask2], data2['flux'][~oot_mask2], ".g", label="data2", zorder=-1000)
plt.plot((data3_fold*24)[~oot_mask3], data3['flux'][~oot_mask3], ".r", label="data3", zorder=-1000)
plt.plot((data4_fold*24)[~oot_mask4], data4['flux'][~oot_mask4], ".b", label="data4", zorder=-1000)

In [ ]:
# fitemos un pol grado 2 a los datos oot
# separando por bandas y fecha del transito
z_B1 = np.polyfit((data1_fold*24)[~oot_mask1], data1['flux'][~oot_mask1], 2) 
z_B2 = np.polyfit((data2_fold*24)[~oot_mask2], data2['flux'][~oot_mask2], 2)
z_z1 = np.polyfit((data3_fold*24)[~oot_mask3], data3['flux'][~oot_mask3], 2)
z_z2 = np.polyfit((data4_fold*24)[~oot_mask4], data4['flux'][~oot_mask4], 2)

def pol_B1(x):
    return np.asarray(z_B1[0]*x**2 + z_B1[1]*x + z_B1[2])
def pol_B2(x):
    return np.asarray(z_B2[0]*x**2 + z_B2[1]*x + z_B2[2])
def pol_z1(x):
    return np.asarray(z_z1[0]*x**2 + z_z1[1]*x + z_z1[2])
def pol_z2(x):
    return np.asarray(z_z2[0]*x**2 + z_z2[1]*x + z_z2[2])

In [ ]:
plt.figure(figsize=(10, 5))
plt.plot((data1_fold*24)[~oot_mask1], data1['flux'][~oot_mask1], ".r", label="data1", zorder=-1000)
plt.plot((data2_fold*24)[~oot_mask2], data2['flux'][~oot_mask2], ".g", label="data2", zorder=-1000)
plt.plot((data3_fold*24)[~oot_mask3], data3['flux'][~oot_mask3], ".k", label="data3", zorder=-1000)
plt.plot((data4_fold*24)[~oot_mask4], data4['flux'][~oot_mask4], ".b", label="data4", zorder=-1000)
x_plot_B1 = np.linspace(np.min((data1_fold*24)[~oot_mask1]), np.max((data1_fold*24)[~oot_mask1]), 1000) 
x_plot_B2 = np.linspace(np.min((data2_fold*24)[~oot_mask2]), np.max((data2_fold*24)[~oot_mask2]), 1000)
x_plot_z1 = np.linspace(np.min((data3_fold*24)[~oot_mask3]), np.max((data3_fold*24)[~oot_mask3]), 1000)
x_plot_z2 = np.linspace(np.min((data4_fold*24)[~oot_mask4]), np.max((data4_fold*24)[~oot_mask4]), 1000)
plt.plot(x_plot_B1, pol_B1(x_plot_B1), color="r", linestyle="--")
plt.plot(x_plot_B2, pol_B2(x_plot_B2), color="g", linestyle="--")
plt.plot(x_plot_z1, pol_z1(x_plot_z1), color="k", linestyle="--")
plt.plot(x_plot_z2, pol_z2(x_plot_z2), color="b", linestyle="--")

#plt.savefig('ajuste_phot_1199.png',dpi=200,bbox_inches='tight', facecolor='white')

In [ ]:
#replot dividing by polynomial
plt.figure(figsize=(8, 5))
# Plot the data
plt.plot(data1_fold*24, (data1['flux']/pol_B1(data1_fold*24)-1)*1e3, 'o', ms=3, color='r', zorder=1000, alpha=0.8)
plt.plot(data2_fold*24, (data2['flux']/pol_B2(data2_fold*24)-1)*1e3, 'o', ms=3, color='g', zorder=1000, alpha=0.8)
plt.plot(data3_fold*24, (data3['flux']/pol_z1(data3_fold*24)-1)*1e3, 'o', ms=3, color='k', zorder=1000, alpha=0.8)
plt.plot(data4_fold*24, (data4['flux']/pol_z2(data4_fold*24)-1)*1e3, 'o', ms=3, color='b', zorder=1000, alpha=0.8)
#plt.legend(fontsize=14, loc=4)
plt.ylabel(u'$\Delta Flux$ [‰]')
plt.xlabel("Time since transit [h]")
plt.yticks([-12,-10,-8, -6, -4, -2,0,2,4,6,8,10,12])
plt.xlim(-0.10*24, 0.10*24)
# horizontal line at 0
plt.axhline(0, color="k", linestyle=":")
plt.ylim(-12, 12)
plt.tight_layout()
#plt.savefig('new_phot_1199_keplercam.png',dpi=300,bbox_inches='tight', facecolor='white')
plt.show()


In [ ]:
# datos de keplercam finales
y_conc_B = np.concatenate(((data1['flux']/pol_B1(data1_fold*24)-1)*1e3, 
                          (data2['flux']/pol_B2(data2_fold*24)-1)*1e3), axis=None)
y_conc_z = np.concatenate(((data3['flux']/pol_z1(data3_fold*24)-1)*1e3, 
                          (data4['flux']/pol_z2(data4_fold*24)-1)*1e3), axis=None)
yerr_conc_B = np.concatenate((data1['flux_err'], data2['flux_err']), axis=None)
yerr_conc_z = np.concatenate((data3['flux_err'], data4['flux_err']), axis=None)

x_keplercam_B = np.concatenate((data1['time'], data2['time']), axis=None)
x_keplercam_z = np.concatenate((data3['time'], data4['time']), axis=None)
y_keplercam_B = y_conc_B
yerr_keplercam_B = yerr_conc_B
y_keplercam_z = y_conc_z
yerr_keplercam_z = yerr_conc_z

In [ ]:
# new detrending ?

In [ ]:
data1_ = pd.read_table('data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0213_KeplerCam_B.tbl', sep='\s+')
data2_ = pd.read_table('data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0413_KeplerCam_B_measurements.tbl', sep='\s+')
data3_ = pd.read_table('data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0213_KeplerCam_z.tbl', sep='\s+')
data4_ = pd.read_table('data/photom/TOI1199/KEPLERCAM/TIC99869022.01_UT2021.0413_KeplerCam_z_measurements.tbl', sep='\s+')

In [ ]:
data1_['flux'] = data1_['rel_flux_T1_dn']*1e3-1e3
data2_['flux'] = data2_['rel_flux_T1_dn']*1e3-1e3
data3_['flux'] = data3_['rel_flux_T1_dn']*1e3-1e3
data4_['flux'] = data4_['rel_flux_T1_dn']*1e3-1e3

data1_['flux_err'] = data1_['rel_flux_err_T1_dn']*1e3
data2_['flux_err'] = data2_['rel_flux_err_T1_dn']*1e3
data3_['flux_err'] = data3_['rel_flux_err_T1_dn']*1e3
data4_['flux_err'] = data4_['rel_flux_err_T1_dn']*1e3

data1_['time'] = data1_['BJD_TDB']-2457000-ref_time
data2_['time'] = data2_['BJD_TDB']-2457000-ref_time
data3_['time'] = data3_['BJD_TDB']-2457000-ref_time
data4_['time'] = data4_['BJD_TDB']-2457000-ref_time

In [ ]:
# Plot the folded data, con los parametros del BLS
data1_fold_ = (data1_['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data2_fold_ = (data2_['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data3_fold_ = (data3_['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data4_fold_ = (data4_['time'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period

plt.figure(figsize=(10, 5))
plt.plot(data1_fold_*24, data1_['flux'], ".k", label="data1", zorder=-1000)
plt.plot(data2_fold_*24, data2_['flux'], ".k", label="data2", zorder=-1000)
plt.plot(data3_fold_*24, data3_['flux'], ".k", label="data3", zorder=-1000)
plt.plot(data4_fold_*24, data4_['flux'], ".k", label="data4", zorder=-1000)
plt.plot(data1_fold*24, (data1['flux']/pol_B1(data1_fold*24)-1)*1e3, 'o', ms=3, color='r', zorder=1000, alpha=0.8)
plt.plot(data2_fold*24, (data2['flux']/pol_B2(data2_fold*24)-1)*1e3, 'o', ms=3, color='r', zorder=1000, alpha=0.8)
plt.plot(data3_fold*24, (data3['flux']/pol_z1(data3_fold*24)-1)*1e3, 'o', ms=3, color='r', zorder=1000, alpha=0.8)
plt.plot(data4_fold*24, (data4['flux']/pol_z2(data4_fold*24)-1)*1e3, 'o', ms=3, color='r', zorder=1000, alpha=0.8)
plt.ylabel("de-trended flux (ppt)")
plt.xlabel('time since transit (hours)')
plt.title("TOI1199")
#plt.savefig('new_phot_1199.png',dpi=200,bbox_inches='tight', facecolor='white')
plt.show()

### MuSCAT2

In [ ]:
# filtros griz sloan: g', r', i', z'
#       Wavelength	Effective Central Wavelength	FWHM	
# g'	401-550nm	475.4nm	                        138.7nm	
# r'	562-695nm	620.4nm	                        124.0nm	
# i'	695-844nm	769.8nm	                        130.3nm	
# z'	>820nm	    900.0nm	                        100.0nm

In [ ]:
data5 = pd.read_table('./data/photom/TOI1199/TOI1199-01_MuSCAT2/TOI1199-01_20210427_zs_TCS_MuSCAT2_Detrended.dat', names=['bjd', 'flux', 'flux_err'], sep='\s+')
data6 = pd.read_table('./data/photom/TOI1199/TOI1199-01_MuSCAT2/TOI1199-01_20210427._g_TCS_MuSCAT2_Detrended.dat', names=['bjd', 'flux', 'flux_err'], sep='\s+')
data7 = pd.read_table('./data/photom/TOI1199/TOI1199-01_MuSCAT2/TOI1199-01_20210427._i_TCS_MuSCAT2_Detrended.dat', names=['bjd', 'flux', 'flux_err'], sep='\s+')
data8 = pd.read_table('./data/photom/TOI1199/TOI1199-01_MuSCAT2/TOI1199-01_20210427._r_TCS_MuSCAT2_Detrended.dat', names=['bjd', 'flux', 'flux_err'], sep='\s+')


In [ ]:
print(np.mean(data7['flux']-1))
print(np.mean(data7['flux']/np.median(data7['flux'])-1))

In [ ]:
# Convert time to the same reference time as tess and flux to ppt
data5['bjd'] = data5['bjd']-2457000-ref_time
data6['bjd'] = data6['bjd']-2457000-ref_time
data7['bjd'] = data7['bjd']-2457000-ref_time
data8['bjd'] = data8['bjd']-2457000-ref_time
data5['flux'] = np.ascontiguousarray(1e3 * (data5['flux'] - 1.0), dtype=np.float64) 
data6['flux'] = np.ascontiguousarray(1e3 * (data6['flux'] - 1.0), dtype=np.float64)
data7['flux'] = np.ascontiguousarray(1e3 * (data7['flux'] - 1.0), dtype=np.float64)
data8['flux'] = np.ascontiguousarray(1e3 * (data8['flux'] - 1.0), dtype=np.float64)
data5['flux_err'] = np.ascontiguousarray(1e3 * (data5['flux_err']), dtype=np.float64) 
data6['flux_err'] = np.ascontiguousarray(1e3 * (data6['flux_err']), dtype=np.float64)
data7['flux_err'] = np.ascontiguousarray(1e3 * (data7['flux_err']), dtype=np.float64)
data8['flux_err'] = np.ascontiguousarray(1e3 * (data8['flux_err']), dtype=np.float64)

In [ ]:
# Texp?
print(np.median(np.diff(data8['bjd']))*24*60*60)
texp_muscat = None

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(data5.bjd, data5.flux, 'o', label='z', ms=2)
plt.plot(data6.bjd, data6.flux, 'o', label='z', ms=2)
plt.plot(data7.bjd, data7.flux, 'o', label='z', ms=2)
plt.plot(data8.bjd, data8.flux, 'o', label='z', ms=2)
plt.show()

In [ ]:
# Plot the folded data, con los parametros del BLS
data5_fold = (data5['bjd'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data6_fold = (data6['bjd'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data7_fold = (data7['bjd'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period
data8_fold = (data8['bjd'] - bls_t0 + 0.5 * bls_period) % bls_period - 0.5 * bls_period

muscat_time = pd.concat([data5_fold, data6_fold, data7_fold, data8_fold], ignore_index=True)
# datos finales muscat
x_muscat_zs_ = np.ascontiguousarray(data5['bjd'], dtype=np.float64)
x_muscat_g_ = np.ascontiguousarray(data6['bjd'], dtype=np.float64)
x_muscat_i_ = np.ascontiguousarray(data7['bjd'], dtype=np.float64)
x_muscat_r_ = np.ascontiguousarray(data8['bjd'], dtype=np.float64)
y_muscat_zs_ = np.ascontiguousarray(data5['flux'], dtype=np.float64)
y_muscat_g_ = np.ascontiguousarray(data6['flux'], dtype=np.float64)
y_muscat_i_ = np.ascontiguousarray(data7['flux'], dtype=np.float64)
y_muscat_r_ = np.ascontiguousarray(data8['flux'], dtype=np.float64)
yerr_muscat_zs_ = np.ascontiguousarray(data5['flux_err'], dtype=np.float64)
yerr_muscat_g_ = np.ascontiguousarray(data6['flux_err'], dtype=np.float64)
yerr_muscat_i_ = np.ascontiguousarray(data7['flux_err'], dtype=np.float64)
yerr_muscat_r_ = np.ascontiguousarray(data8['flux_err'], dtype=np.float64)

plt.figure(figsize=(10, 5))
plt.plot(data5_fold*24, y_muscat_zs_, '.', label='Muscat2_zs', alpha=0.6, zorder=100)
plt.errorbar(data5_fold*24, y_muscat_zs_, yerr=yerr_muscat_zs_, fmt="none", alpha=0.2, color='#777777', capsize=1, zorder=-100)
plt.plot(data6_fold*24, y_muscat_g_, '.', label='Muscat2_g', alpha=0.6, zorder=100)
plt.errorbar(data6_fold*24, y_muscat_g_, yerr=yerr_muscat_g_, fmt="none", alpha=0.2, color='#777777', capsize=1, zorder=-100)
plt.plot(data7_fold*24, y_muscat_i_, '.', color='red', label='Muscat2_i', alpha=0.6, zorder=100)
plt.errorbar(data7_fold*24, y_muscat_i_, yerr=yerr_muscat_i_, fmt="none", alpha=0.2, color='#777777', capsize=1, zorder=-100)
plt.plot(data8_fold*24, y_muscat_r_, '.', label='Muscat2_r', alpha=0.6, zorder=100)
plt.errorbar(data8_fold*24, y_muscat_r_, yerr=yerr_muscat_r_, fmt="none", alpha=0.2, color='#777777', capsize=1, zorder=-100)

plt.ylabel("de-trended flux (ppt)")
plt.xlabel('time since transit (hours)')
#plt.xlim(-0.10*24, 0.10*24)
plt.ylim(-25,25)
plt.title("TOI1199")
#plt.savefig('new_phot_1199.png',dpi=200,bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# mask the first half of the transit
x_muscat_zs = x_muscat_zs_[data5_fold*24>0.10]
x_muscat_g = x_muscat_g_[data6_fold*24>0.10]
x_muscat_i = x_muscat_i_[data7_fold*24>0.10]
x_muscat_r = x_muscat_r_[data8_fold*24>0.10]
y_muscat_zs = y_muscat_zs_[data5_fold*24>0.10]
y_muscat_g = y_muscat_g_[data6_fold*24>0.10]
y_muscat_i = y_muscat_i_[data7_fold*24>0.10]
y_muscat_r = y_muscat_r_[data8_fold*24>0.10]
yerr_muscat_zs = yerr_muscat_zs_[data5_fold*24>0.10]
yerr_muscat_g = yerr_muscat_g_[data6_fold*24>0.10]
yerr_muscat_i = yerr_muscat_i_[data7_fold*24>0.10]
yerr_muscat_r = yerr_muscat_r_[data8_fold*24>0.10]

print(len(x_muscat_zs), len(y_muscat_zs), len(yerr_muscat_zs))
print(len(x_muscat_g), len(y_muscat_g), len(yerr_muscat_g))
print(len(x_muscat_i), len(y_muscat_i), len(yerr_muscat_i))
print(len(x_muscat_r), len(y_muscat_r), len(yerr_muscat_r))


# LC dataset

In [ ]:
from collections import OrderedDict

# datasets = OrderedDict(
#     [
#         ("tess_2min", [x, y, yerr, texp]),
#         ("tess_30min", [xlong, ylong, yerrlong, texp_long]),
#         ("keplercam_B", [x_keplercam_B, y_keplercam_B, yerr_keplercam_B, texp_keplercam]),
#         ('muscat_g', [x_muscat_g, y_muscat_g, yerr_muscat_g, texp_muscat]),
#         ('muscat_r', [x_muscat_r, y_muscat_r, yerr_muscat_r, texp_muscat]),
#         ('muscat_i', [x_muscat_i, y_muscat_i, yerr_muscat_i, texp_muscat]),
#         ("keplercam_z", [x_keplercam_z, y_keplercam_z, yerr_keplercam_z, texp_keplercam]),
#         ('muscat_zs', [x_muscat_zs, y_muscat_zs, yerr_muscat_zs, texp_muscat]),
#     ]
# )
datasets = OrderedDict(
    [
        ("tess_2min", [x, y, yerr, texp]),
        ("tess_30min", [xlong, ylong, yerrlong, texp_long]),
        ("keplercam_B", [x_keplercam_B, y_keplercam_B, yerr_keplercam_B, texp_keplercam]),
        ("keplercam_z", [x_keplercam_z, y_keplercam_z, yerr_keplercam_z, texp_keplercam]),
    ]
)

In [ ]:
for n, (name, (x_, y_, yerr_, texp_)) in enumerate(datasets.items()):
    print(n, name, len(x_), len(y_), len(yerr_), type(x_), type(y_), type(yerr_))

In [ ]:
fig, axes = plt.subplots(1, len(datasets), sharey=True, figsize=(15, 5))

for i, (name, (t, y, _, _)) in enumerate(datasets.items()):
    ax = axes[i]
    ax.plot(t, y, "k", lw=0.75, label=name)
    ax.set_xlabel("time")
    ax.set_title(name, fontsize=14)

    x_mid = 0.5 * (t.min() + t.max())
    ax.set_xlim(x_mid - 120, x_mid + 120)
axes[0].set_ylim(-10, 10)
fig.subplots_adjust(wspace=0.05)
_ = axes[0].set_ylabel("relative flux [ppt]")

In [ ]:
K_est = xo.estimate_semi_amplitude(bls_period, x_rv, y_rv, yerr_rv, t0s=bls_t0)
print(K_est, "m/s")

In [ ]:
K_est.item()

In [ ]:
0.5*np.log(spoc_depth * 1e-3)

# Joint model

In [ ]:
# These arrays are used as the times/phases where the models are
# evaluated at higher resolution for plotting purposes 
t_rv = np.linspace(x_rv.min() - 5, x_rv.max() + 5, 3000)
phase_lc = np.linspace(-0.3, 0.3, 500)

for i in range(10):
    with pm.Model() as model:
        # Parameters for the stellar properties
        BoundedNormal = pm.Bound(pm.Normal, lower=0, upper=3)
        m_star = BoundedNormal("m_star", mu=M_star[0], sd=M_star[1])
        r_star = BoundedNormal("r_star", mu=R_star[0], sd=R_star[1])

        # Orbital parameters for the planets
        t0 = pm.Normal("t0", mu=bls_t0, sd=1)
        log_period = pm.Normal("log_period", mu=np.log(bls_period), sd=1)

        # b = pmx.UnitUniform("b")
        log_ror = pm.Normal("log_ror", mu=0.5*np.log(spoc_depth * 1e-3), sd=1)
        ror = pm.Deterministic("ror", tt.exp(log_ror))
        r_pl = pm.Deterministic("r_pl", ror * r_star)
        b = xo.ImpactParameter('b', ror)

        logK = pm.Normal("logK", mu=np.log(K_est.item()), sd=2.0)
        K = pm.Deterministic("K", tt.exp(logK))
        #m_pl = pm.Deterministic("m_pl", tt.exp(log_m_pl))
        period = pm.Deterministic("period", tt.exp(log_period))

        # comentar las proximas tres lineas para versión con e=0 fixed
        ecs = pmx.UnitDisk("ecs", testval=np.array([0.01, 0.01]))
        ecc = pm.Deterministic("ecc", tt.sum(ecs**2))
        omega = pm.Deterministic("omega", tt.arctan2(ecs[1], ecs[0]))

        # RV jitter & a quadratic RV trend
        log_sigma_rv = pm.Normal("log_sigma_rv", mu=np.log(np.median(yerr_rv)), sd=1)
        trend = pm.Normal("trend", mu=0, sd=10.0 ** -np.arange(3)[::-1], shape=3)  # [2,1,0]

        # Orbit model
        orbit = xo.orbits.KeplerianOrbit(
            r_star=r_star,
            m_star=m_star,
            period=period,
            t0=t0,
            b=b,
            ecc=ecc,
            omega=omega,
        )

        # Compute the mass and the density of the planet
        m_pl = pm.Deterministic('m_pl', K*tt.sqrt(1-ecc**2)*(period*u.day.to(u.second)*(m_star*u.M_sun.to(u.kg))**2/(2*np.pi*constants.G))**(1/3))
        density_pl = pm.Deterministic('density_pl', m_pl*u.kg.to(u.g)/(4/3*np.pi*(r_pl*u.R_sun.to(u.cm))**3))
        
        a = pm.Deterministic('a', ((period*u.day.to(u.second))**2*constants.G*(m_star*u.M_sun.to(u.kg)+m_pl)/(4*np.pi**2))**(1/3)*u.m.to(u.au))
        teq = pm.Deterministic('teq', teff*(r_star*u.R_sun.to(u.au)/a)**(1/2)*(1/4)**(1/4))

        # Loop over the instruments
        parameters = dict()
        lc_models = dict()
        transit_obs = dict()
        lc_pred = dict()
        
        # The light curve model
        def lc_model(mean, star, r_pl, texp, t):
            return tt.sum(1e3*star.get_light_curve(orbit=orbit, r=r_pl, t=t, texp=texp), axis=-1) + mean
        
        for n, (name, (x_, y_, yerr_, texp_)) in enumerate(datasets.items()):
            # We define the per-instrument parameters in a submodel so that we
            # don't have to prefix the names manually
            
            with pm.Model(name=name, model=model):
                # The flux zero point
                mean = pm.Normal("mean", mu=0.0, sd=5.0)
                # The limb darkening
                # both tess datasets should share the same limb darkening coefficients
                if "tess_30min" in name:
                    star = xo.LimbDarkLightCurve(u_star)
                else:
                    u_star = xo.QuadLimbDark("u_star")
                    star = xo.LimbDarkLightCurve(u_star)

                # Transit jitter
                logs = pm.Normal("logs", mu=np.log(np.median(yerr_)), sd=1)
                
                # Keep track of the parameters for optimization
                parameters[name] = [mean, u_star, logs]

            #lc_model = partial(lc_model, mean, star, ror, texp)
            lc_models[name] = lc_model(mean, star, r_pl, texp_, x_)

            # The likelihood for the light curve
            err_lc = tt.sqrt(yerr_**2 + tt.exp(2*logs)) 
            transit_obs = pm.Normal(f"{name}_obs", mu=lc_models[name], sd=err_lc, observed=y_) 

            # Compute and save the phased light curve models
            pm.Deterministic(f"{name}_lc_pred", tt.sum(star.get_light_curve(orbit=orbit, r=r_pl, t=t0+phase_lc, texp=texp_)*1e3, axis=-1))


        # And a function for computing the full RV model
        def get_rv_model(t, name=""):
            # First the RVs induced by the planet
            vrad = orbit.get_radial_velocity(t, K=tt.exp(logK))
            pm.Deterministic("vrad" + name, vrad)

            # Define the background model
            A = np.vander(t - x_ref, 3) # Generate a Vandermonde matrix with t-x_ref as input vector
            bkg = pm.Deterministic("bkg" + name, tt.dot(A, trend))

            # Sum planet and background to get the full model
            return pm.Deterministic("rv_model" + name, vrad + bkg)


        # Define the model
        rv_model = get_rv_model(x_rv)
        get_rv_model(t_rv, name="_pred")

        # The likelihood for the RVs
        err = tt.sqrt(yerr_rv**2 + tt.exp(2 * log_sigma_rv))
        pm.Normal("obs", mu=rv_model, sd=err, observed=y_rv)

        # Optimize the model
        map_soln = model.test_point
        for name in datasets:
            map_soln = pmx.optimize(map_soln, parameters[name])
        for name in datasets:
            map_soln = pmx.optimize(map_soln, parameters[name] + [logK, ecs, b, log_period]) # [logK, ecs, b, log_period]
            map_soln = pmx.optimize(map_soln, parameters[name] + [log_sigma_rv, trend])
        map_soln = pmx.optimize(map_soln)

        extras = dict()
        for name in datasets:
            extras[name] = pmx.eval_in_model(lc_models[name], map_soln)

        # Do some sigma clipping
        num = dict((name, len(datasets[name][0])) for name in datasets)
        clipped = dict()
        masks = dict()
        for name in datasets:
            mdl = pmx.eval_in_model(lc_models[name], map_soln)
            resid = datasets[name][1] - mdl
            sigma = np.sqrt(np.median((resid - np.median(resid)) ** 2))
            masks[name] = np.abs(resid - np.median(resid)) < 7 * sigma
            clipped[name] = num[name] - masks[name].sum()
            print(f"Sigma clipped {clipped[name]} {name} light curve points")

        if all(c < 10 for c in clipped.values()):
            break

        else:
            for name in datasets:
                datasets[name][0] = datasets[name][0][masks[name]]
                datasets[name][1] = datasets[name][1][masks[name]]
                datasets[name][2] = datasets[name][2][masks[name]]

In [ ]:
model.free_RVs

MAP rv model

In [ ]:
# We plot the initial model:
def plot_rv_curve(soln):
    fig, axes = plt.subplots(2, 1, figsize=(16, 8), sharex=True)
    ax = axes[0]
    ax.errorbar(x_rv, y_rv, yerr=yerr_rv, fmt=".k")
    ax.plot(t_rv, soln['vrad_pred'], "--k", alpha=0.5)
    ax.plot(t_rv, soln['bkg_pred'], ":k", alpha=0.5)
    ax.plot(t_rv, soln['rv_model_pred'], label="model")
    ax.legend(fontsize=10)
    ax.set_xlabel("time [days]")
    ax.set_ylabel("radial velocity [m/s]")
    
    ax = axes[1]
    err = np.sqrt(yerr_rv**2 + np.exp(2 * soln["log_sigma_rv"]))
    ax.errorbar(x_rv, y_rv - soln["rv_model"], yerr=err, fmt=".k")
    ax.axhline(0, color="k", lw=1)
    ax.set_ylabel("residuals [m/s]")
    ax.set_xlim(t_rv.min(), t_rv.max())
    #ax.set_xlim(2200,2300)
    ax.set_xlabel("time [days]")

_ = plot_rv_curve(map_soln)

MAP LCs models

In [ ]:
fig, axes = plt.subplots(len(datasets), sharex=True, sharey=True, figsize=(8, 12))

for n, name in enumerate(datasets):
    ax = axes[n]

    x, y = datasets[name][:2]

    period = map_soln["period"]
    folded = (x - map_soln["t0"] + 0.5 * period) % period - 0.5 * period
    m = np.abs(folded) < 0.2
    ax.plot(
        folded[m],
        (y  - map_soln[f"{name}_mean"])[m],
        ".k",
        alpha=0.3,
        mec="none",
    )
    ax.plot(
        phase_lc, map_soln[name+'_lc_pred'] , f"C{n}", label=name
    )

    ax.annotate(
        name,
        xy=(1, 0),
        xycoords="axes fraction",
        va="bottom",
        ha="right",
        xytext=(-3, 3),
        textcoords="offset points",
        fontsize=14,
    )

axes[-1].set_xlim(-0.15, 0.15)
axes[-1].set_xlabel("time since transit [days]")
for ax in axes:
    ax.set_ylabel("relative flux [ppt]")

#plt.savefig('lcs_fits.png', dpi=200, bbox_inches='tight')

In [ ]:
# pm.model_to_graphviz(model)

In [ ]:
len(model.free_RVs)

# Sampling

In [ ]:
with model:
    trace = pmx.sample(
        tune=4000, #1500,
        draws=4000, #1000,
        start=map_soln,
        cores=2, #4
        chains=2,
        target_accept=0.95,
        return_inferencedata=True,
        random_seed=[203771098, 203775000],
        init="adapt_full",
   )

In [ ]:
import arviz as az
# # version con e libre
az.to_netcdf(trace, 'trace.toi1199_report.save')
# trace = az.from_netcdf('trace.toi1199_final.save')

# # version con e=0 fixed
# # az.to_netcdf(trace, 'trace.toi1199_noecc.save')
# # trace = az.from_netcdf('trace.toi1199_noecc.save')

In [ ]:
#trace.posterior.data_vars
# print only the names
for n, var in enumerate(trace.posterior.data_vars):
    if n>15:
        print(n, var)


Summary stats

In [ ]:
summary = az.summary(trace, stat_funcs={'median': np.median, 'std':np.std}, hdi_prob=0.997, round_to=6,
           var_names=[
               't0',
               'logK',
               'log_period',
               'log_ror',
               'log_sigma_rv',
               'trend',
               'tess_2min_mean',
               'tess_2min_logs',
               'tess_30min_mean',
               'tess_30min_logs',
               'keplercam_B_mean',
               'keplercam_B_logs',
               'keplercam_z_mean',
               'keplercam_z_logs',
               'm_star',
               'r_star',
               'ror',
               'r_pl',
               'm_pl',
               'density_pl',
               'a',
               'teq',
               'b',
               'K',
               'period',
               'ecs',
               'ecc',
               'omega',
               'tess_2min_u_star',
               'keplercam_B_u_star',
               'keplercam_z_u_star',
           ])

In [ ]:
print(summary.ess_bulk.mean())
print(summary.ess_tail.mean())

In [ ]:
summary

In [ ]:
#plot limb darkening coefficient vs radius
plt.plot(trace.posterior["tess_2min_u_star"].stack(sample=("chain", "draw"))[0],
        trace.posterior["r_pl"].stack(draws=("chain", "draw")),'.',alpha=0.15, label="q$_1$ vs R$_P$")
plt.plot(trace.posterior["tess_2min_u_star"].stack(sample=("chain", "draw"))[1],
        trace.posterior["r_pl"].stack(draws=("chain", "draw")),'.k',alpha=0.15, label="q$_2$ vs R$_P$")
plt.legend(loc='best', fontsize=11)
plt.xlabel("LD coefficient")
plt.ylabel("R$_P$ (R$_J$)")
plt.title("TOI 1199", fontsize=13)
plt.savefig("toi1199_ld_vs_rp.png", dpi=300, bbox_inches='tight')

In [ ]:
az.plot_posterior(trace, var_names=['r_pl', 'b', 'm_pl', 'density_pl', 'ecc'])
#plt.savefig('toi1199_ustar_posteriors.png', dpi=300, bbox_inches='tight')

In [ ]:
az.plot_posterior(trace, var_names=['ecc'], hdi_prob=0.997)
plt.xlim(0, 1)
plt.show()

Parámetros

In [ ]:
import math
from uncertainties import ufloat
from uncertainties.umath import *

radio = ufloat((trace.posterior["r_pl"].median().item()*u.R_sun).to(u.R_jup).value, (trace.posterior["r_pl"].std().item()*u.R_sun).to(u.R_jup).value)
masa = ufloat((trace.posterior["m_pl"].median().item()*u.kg).to(u.M_jup).value, (trace.posterior["m_pl"].std().item()*u.kg).to(u.M_jup).value)
densidad = ufloat(trace.posterior['density_pl'].median().item(), trace.posterior['density_pl'].std().item())
b = ufloat(trace.posterior["b"].median().item(), trace.posterior["b"].std().item())
K = ufloat(trace.posterior["K"].median().item(), trace.posterior["K"].std().item())
eccen = ufloat(trace.posterior["ecc"].median().item(), trace.posterior["ecc"].std().item())
omega = ufloat(trace.posterior["omega"].median().item(), trace.posterior["omega"].std().item())
periodo = ufloat(trace.posterior['period'].median().item(),trace.posterior['period'].std().item())
t_0 = ufloat(trace.posterior['t0'].median().item()+ref_time, trace.posterior['t0'].std().item())
mstar = ufloat(trace.posterior['m_star'].median().item()*u.M_sun.to(u.kg), trace.posterior['m_star'].std().item()*u.M_sun.to(u.kg))
rstar = ufloat(trace.posterior['r_star'].median().item()*u.R_sun.to(u.au), trace.posterior['r_star'].std().item()*u.R_sun.to(u.au))
smaxis = ufloat(trace.posterior['a'].median().item(), trace.posterior['a'].std().item())
tequilibrium = ufloat(trace.posterior['teq'].median().item(), trace.posterior['teq'].std().item())

print("Rp = {r:.3f} Rj".format(r=radio))
print("Mp = {m:.3f} Mj".format(m=masa))
print("Density = {d:.3f} g/cm^3".format(d=densidad))
print("b = {b:.3f}".format(b=b))
print("e = {e:.6f}".format(e=eccen))
print("omega = {w:.6f}".format(w=omega))
print("P = {p:.6f} days".format(p=periodo))
print("Epoca = {e:.5f}".format(e=t_0))
print('K = {k:.3f} m/s'.format(k=K))
print('a = {a:.4f} AU'.format(a=smaxis))
print('Teq = {t:.3f} K'.format(t=tequilibrium))

In [ ]:
# estimate the expected jitter value from log_rhk according to Santos et al 2000
# this is the relation for G dwarfs
def sjitter(log_rhk):
    rhk = 10**(log_rhk)
    r5 = 10**5 * rhk
    s_jitter = r5**(0.55) * 7.9
    return s_jitter

print(sjitter(ufloat(-5.0, 0.2)))

In [ ]:
0.018/0.233*100

In [ ]:
# in the same plot show two histograms of radio_posterior and mpl_posterior
fig, ax = plt.subplots(3, figsize=(8, 9))
fig.subplots_adjust(hspace=0.29)
#ax[0].set_title('TOI-1199 b', fontsize=14)
ax[0].hist(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup), 
            bins=50, density=True, histtype='step', color='k', label='R$_\mathrm{P}$ posterior')
ax[0].hist(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup), 
            bins=50, density=True, histtype='stepfilled', color='k', alpha=0.1)
ax[1].hist(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup), 
            bins=50, density=True,histtype='step', color='k', label='M$_\mathrm{P}$ posterior')
ax[1].hist(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup), 
            bins=50, density=True,histtype='stepfilled', color='k', alpha=0.1)
ax[2].hist(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy(), 
            bins=50, density=True, histtype='step', color='k')
ax[2].hist(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy(), 
            bins=50, density=True, histtype='stepfilled', color='k', alpha=0.1, zorder=10)
#ax[0].legend(markerscale=0.,frameon=False, prop=dict(family='serif', size=10))
#ax[1].legend(frameon=False, prop=dict(family='serif', size=10))
#ax[2].legend(frameon=False, prop=dict(family='serif', size=10))

ax[0].set_xlabel('R (R$_\mathrm{J}$)', fontsize=12, labelpad=0.5)
ax[1].set_xlabel('Mp (M$_\mathrm{J}$)', fontsize=12, labelpad=0.5)
ax[2].set_xlabel('Density ($\mathrm{g}\,\mathrm{cm}^{-3}$)', fontsize=12, labelpad=0.5)

for i in range(3):
    ax[i].set_ylabel('Relative frequency', fontsize=12)
# add the median and 1-sigma error bars at the top left
ax[0].errorbar(np.median(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup)), 1, 
                xerr=np.std(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup)), 
                color='k', capsize=5, capthick=1.5)
ax[1].errorbar(np.median(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup)), 2, 
                xerr=np.std(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup)), 
                color='k', capsize=5, capthick=1.5)
ax[2].errorbar(np.median(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy()), 0.8, 
                xerr=np.std(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy()), 
                color='k', capsize=5, capthick=1.5)
# show the numbers in the plot
ax[0].text(0.41, 0.13, '{:.2f} $\pm$ {:.2f}'.format(np.median(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup)), 
            np.std(trace.posterior['r_pl'].stack(sample=("chain", "draw")).to_numpy()*u.R_sun.to(u.R_jup))), transform=ax[0].transAxes,
            fontsize=9, weight='regular', fontfamily='serif')
ax[1].text(0.43, 0.13, '{:.2f} $\pm$ {:.2f}'.format(np.median(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup)), 
            np.std(trace.posterior['m_pl'].stack(sample=("chain", "draw")).to_numpy()*u.kg.to(u.M_jup))), transform=ax[1].transAxes,
            fontsize=9, weight='regular', fontfamily='serif')
ax[2].text(0.325, 0.14, '{:.2f} $\pm$ {:.2f}'.format(np.median(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy()), 
            np.std(trace.posterior['density_pl'].stack(sample=("chain", "draw")).to_numpy())), transform=ax[2].transAxes, 
            fontsize=9, weight='regular', fontfamily='serif')
for i in range(3):
    ax[i].tick_params(axis='y', which='both', left=False, right=False, direction='inout', labelsize=12)
    ax[i].tick_params(axis='x', which='both', bottom=True, top=False, direction='inout', labelsize=12)
    ax[i].set_yticks([])
ax[2].set_xticks(np.arange(0.1, 0.95, 0.1))
ax[2].set_xlim(0, 0.9)
#plt.savefig('posterior_density_1199.png', dpi=300, bbox_inches='tight', facecolor='w')
plt.show()

In [ ]:
logsigma_rv_mean = trace.posterior["log_sigma_rv"].median().item()
logsigma_rv = ufloat(logsigma_rv_mean, trace.posterior["log_sigma_rv"].std().item())
rv_jitter_ = np.e**logsigma_rv
tess_2min_jitter_median = trace.posterior['tess_2min_logs'].median().item()
tess_2min_jitter = ufloat(tess_2min_jitter_median, trace.posterior['tess_2min_logs'].std().item())
tess_2min_jitter_val = np.e**tess_2min_jitter
tess_30min_jitter_median = trace.posterior['tess_30min_logs'].median().item()
tess_30min_jitter = ufloat(tess_30min_jitter_median, trace.posterior['tess_30min_logs'].std().item())
tess_30min_jitter_val = np.e**tess_30min_jitter
keplercam_B_jitter_median = trace.posterior['keplercam_B_logs'].median().item()
keplercam_B_jitter = ufloat(keplercam_B_jitter_median, trace.posterior['keplercam_B_logs'].std().item())
keplercam_B_jitter_val = np.e**keplercam_B_jitter
keplercam_z_jitter_median = trace.posterior['keplercam_z_logs'].median().item()
keplercam_z_jitter = ufloat(keplercam_z_jitter_median, trace.posterior['keplercam_z_logs'].std().item())
keplercam_z_jitter_val = np.e**keplercam_z_jitter
print("RV jitter = ", rv_jitter_)
print("TESS 2min jitter = ", tess_2min_jitter_val)
print("TESS 30min jitter = ", tess_30min_jitter_val)
print("Keplercam B jitter = ", keplercam_B_jitter_val)
print("Keplercam z jitter = ", keplercam_z_jitter_val)


In [ ]:
# resultados sin los sectores de 30 min
# Rp = 0.945+/-0.034 Rj
# b = 0.856+/-0.013
# e = 0.027714+/-0.026523
# omega = 0.841376+/-1.796590
# P = 3.671472+/-0.000012 days
# Epoca = 2420.53748+/-0.00046
# a = 0.0492+/-0.0003 au
# Teq = 1495.749+/-28.132 K
# m_pl = 0.233+/-0.018 Mj
# K = 27.517+/-2.090 m/s
# densidad = 0.342+/-0.045 g/cm^3

In [ ]:
from arviz.utils import Numba
Numba.disable_numba()
Numba.numba_flag

_ = az.plot_trace(trace, var_names=['r_pl', 'm_pl','density_pl', 'b'] 
                                    , compact=False, figsize=(12,20)) 

#plt.savefig('trace_1199.png', dpi=300, bbox_inches='tight')

In [ ]:
trace3 = trace.copy()
trace3.posterior['r_pl'] = trace3.posterior['r_pl']*u.R_sun.to(u.R_jup)
trace3.posterior['m_pl'] = trace3.posterior['m_pl']*u.kg.to(u.M_jup)
trace3.posterior['t0'] = trace3.posterior['t0']+2457000+ref_time

## Cornerplot

In [ ]:
CORNER_KWARGS = dict(
    smooth=0.5,
    plot_density=True,
    plot_datapoints=True,
    fill_contours=False,
    #levels=(1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)),
    levels=(0.16,0.5,0.84),
    max_n_ticks=3
)

In [ ]:
from cProfile import label
import corner
from matplotlib.ticker import ScalarFormatter
formatter = ScalarFormatter()

figure = corner.corner(trace3, var_names=['period', 'K', 'm_pl', 'r_pl', 'b', 'ecc', 'omega', 'm_star', 'r_star'],
                        labels=["P (d)", "K (m s$^{-1}$)", "M$_P$ (M$_J$)", "R$_P$ (R$_J$)", "b", "e", "$\omega~$(rad)","M$_\star$ (M$_\odot$)","R$_\star$ (R$_\odot$)"], label_kwargs={'fontsize':20},
                        show_titles=False, title_kwargs={'fontsize':20, 'pad':9.0}, titles=['P', 'K', 'M\_P','R\_P','b','e','$\omega$','M\_$\star$','R\_$\star$'], divergences=False, plot_contours=True,
                        labelpad=0.2, color='k', **CORNER_KWARGS)
for i,ax in enumerate(figure.get_axes()):
    ax.tick_params(axis='both', pad=1, direction='in', labelbottom=True, labelleft=True, left=True, bottom=True)
    #ax.ticklabel_format(axis='both', style='plain', scilimits=None, useOffset=None, useLocale=None, useMathText=None)
    if i == 71 or i == 72 or i == 73:
        ax.xaxis.set_major_formatter(formatter)

#plt.savefig('corner_1199_paper.png', dpi=400, bbox_inches='tight', facecolor='w')
plt.show()

## LC Phase plot

In [ ]:
colormap1 = ['#FFEE58', '#FFEE58', '#5C6BC0', '#4FC3F7', '#FFB74D', '#FF8A65', '#F44336', '#A1887F']

In [ ]:
flat_samps = trace.posterior.stack(sample=("chain", "draw"))

# Get the posterior median orbital parameters
p_ = np.median(flat_samps["period"])
p_err = np.std(flat_samps["period"])
t0_ = np.median(flat_samps["t0"])
t0_err = np.std(flat_samps["t0"])
tess_2min_mean = np.median(flat_samps["tess_2min_mean"])
tess_30min_mean = np.median(flat_samps["tess_30min_mean"])
keplercam_B_mean = np.median(flat_samps["keplercam_B_mean"])
keplercam_z_mean = np.median(flat_samps["keplercam_z_mean"])

In [ ]:
fig, axes = plt.subplots(len(datasets), sharex=True, sharey=True, figsize=(6, 10))
plt.subplots_adjust(hspace=0.0)
# add a big axis, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Time since transit [h]")
plt.ylabel(u'$\Delta Flux$ [‰]')
for n, name in enumerate(datasets):
    ax = axes[n]
    # plot the folded transits
    x, y, yerr = datasets[name][:3]
    folded = (x - t0_ + 0.5 * p_) % p_ - 0.5 * p_
    mean = np.median(flat_samps[f"{name}_mean"])
    print(mean)
    ax.plot(folded*24, (y  - mean), marker='o', ls='none', color=colormap1[n], 
            mec=colormap1[n], mew=1, alpha=1, label=f"{name}", ms=5, zorder=8)
    ax.errorbar(folded*24, (y  - mean), yerr=yerr, fmt='o', color=colormap1[n],
                alpha=0.8, ms=5, zorder=8, elinewidth=0.5, capsize=0)
    # Overplot the phase binned light curve 
    lcc = lk.LightCurve(time=folded, flux=y - mean, flux_err=yerr)
    lcc_binned = lcc.bin(time_bin_size=0.01)
    ax.scatter(lcc_binned['time'].value*24, lcc_binned['flux'].value, marker='o', s=30,  edgecolors='w', linewidths=0.5, c='k', alpha=0.9, zorder=12)
    # overplot the model
    pred = np.percentile(flat_samps[f"{name}_lc_pred"], [16, 50, 84], axis=-1)
    #ax.plot(phase_lc*24, pred[1], color="k", linewidth=1, alpha=1, zorder=11) #f"C{n+1}"
    art = ax.fill_between(phase_lc*24, pred[0], pred[2], color="w", alpha=1, zorder=10)
    art.set_edgecolor("k")
    ax.legend(fontsize=11, loc=3, frameon=False, handlelength=0, handletextpad=0, borderpad=0.5, markerscale=0)
    ax.tick_params(axis='y', which='major', size=4, left=True, right=True, direction='in', labelsize=13)
    ax.tick_params(axis='y', which='minor', size=2, left=True, right=True, direction='in', labelsize=13)
    ax.tick_params(axis='x', which='major', size=4, bottom=False, top=False, direction='in', labelsize=13)
    if n==5:
        ax.tick_params(axis='x', which='major', size=2, bottom=True, top=False, direction='in', labelsize=13)

axes[-1].set_xlim(-0.08*24, 0.08*24)
axes[-1].set_ylim(-10, 5)

#plt.savefig('1199_lcsfit.png', dpi=300, bbox_inches='tight')
plt.show()

## RV Phase plot

In [ ]:
rv_jitter = np.exp(np.median(flat_samps['log_sigma_rv']))
bkg_ = np.median(flat_samps['bkg'].values, axis=-1)
rv_model_ = np.median(flat_samps['rv_model'].values, axis=-1)

# Plot the folded data
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(10, 6), sharex=True, gridspec_kw={'height_ratios': [3,1]})
fig.subplots_adjust(hspace=0.0)
x_fold = (x_rv - t0_ + 0.5 * p_) % p_ - 0.5 * p_
ax1.errorbar(x_fold/p_, y_rv - bkg_, yerr=np.sqrt(yerr_rv**2+rv_jitter**2), fmt="o", color='k', 
            markeredgecolor='k', ecolor='k', elinewidth=1, label='SOPHIE RV\'s', alpha=1) #color='#FBC15E'

# Compute the posterior prediction for the folded RV model 
t_fold = (t_rv - t0_ + 0.5 * p_) % p_ - 0.5 * p_
inds = np.argsort(t_fold)
pred = np.percentile(trace.posterior["vrad_pred"].values[:, :, inds],[16, 50, 84], axis=(0, 1))
ax1.plot(t_fold[inds]/p_, pred[1], color="#FF7F00", label="Model")
art = ax1.fill_between(t_fold[inds]/p_, pred[0], pred[2], color="#FF7F00", alpha=0.5) #color="#988ED5"
art.set_edgecolor("none")

ax1.set_xlim(-0.51, 0.51)
ax1.set_ylim(-58, 70)
ax1.set_ylabel("Radial velocity (m s$^{-1}$)", labelpad=1, fontsize=12)
handles, labels = ax1.get_legend_handles_labels()
ax1.legend(handles[::-1], labels[::-1], fontsize=12, loc='best')
ax1.tick_params(axis='both', which='both', left=True, bottom=True, top=True, right=True, direction='in', labelsize=12)

# Plot the folded residuals
ax2.axhline(y=0, ls='--', color='k', linewidth=1, alpha=0.5)
ax2.errorbar(x_fold/p_, y_rv-rv_model_, yerr=np.sqrt(yerr_rv**2+rv_jitter**2), fmt="o", color='k',
            markeredgecolor='k', ecolor='k', elinewidth=1, alpha=1, label='residuals')
ax2.set_ylabel('Residuals (m s$^{-1}$)', labelpad=1, fontsize=12)
ax2.set_xlabel("Phase", labelpad=1, fontsize=12)
ax2.tick_params(axis='both', which='both', left=True, bottom=True, top=True, right=True, direction='in', labelsize=12)
ax2.set_xlim(-0.51, 0.51)
ax2.set_ylim(-42, 45)
ax2.set_xticks([-0.5, 0, 0.5])
ax2.set_yticks([-30, 0, 30])
#plt.savefig('1199_rvs_fit.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

### Periodograms with residues

In [ ]:
ls_res = LombScargle(data.bjd, y_rv-rv_model_)
frequency_res, power_res = ls_res.autopower(minimum_frequency=0.001, maximum_frequency=2, samples_per_peak=5)
faps_res = ls_res.false_alarm_level(probabilities)  

In [ ]:
from matplotlib.ticker import ScalarFormatter
# plot two figures in a column
fig, axes = plt.subplots(4, 1, sharex=True, sharey=True, figsize=(7, 6))
plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=0)
axes[0].plot(1/frequency, power, color='k', lw=1, zorder=10) 
axes[1].plot(1/frequency_res, power_res, color='k', lw=1, zorder=10)
axes[2].plot(1/frequency_bis, power_bis, color='k', lw=1, zorder=10) 
#axes[3].plot(1/frequency_window, power_window,  color='k', lw=0.8, zorder=10)
axes[3].plot(prange, wf, color='r', lw=1, zorder=10)

# plot the 'bls_period' as a vertical line behind the plot and show in legend
axes[0].axvline(p_, ls='-', lw=1.5, alpha=0.7, color='deepskyblue', label='Transit period', zorder=-10)
axes[1].axvline(p_, ls='-', lw=1.5, alpha=0.7, color='deepskyblue', label='', zorder=-10)
axes[2].axvline(p_, ls='-', lw=1.5, alpha=0.7, color='deepskyblue', label='', zorder=-10)
axes[3].axvline(p_, ls='-', lw=1.5, alpha=0.7, color='deepskyblue', label='', zorder=-10)
# add the aliases as red dashed vertical lines
for i in range(4):
    if i==0:
        axes[i].axvline(aliases[0], lw=1.5, alpha=0.7, ls='dotted', color='deepskyblue', label='1d aliases')
        axes[i].axvline(aliases[2], lw=1.5, alpha=0.7, ls='dotted', color='deepskyblue')
    else:
        axes[i].axvline(aliases[0], lw=1.5, alpha=0.7, ls='dotted', color='deepskyblue', label='')
        axes[i].axvline(aliases[2], lw=1.5, alpha=0.7, ls='dotted', color='deepskyblue')

# overplot the false alarm probabilities
names2 = [faps, faps_res, faps_bis]
for i, names in enumerate(names2):
    # axes[i].axhline(names[0], ls='--', lw=1, alpha=0.8, color='lightgray', label='10% FAP', zorder=-20)
    # axes[i].axhline(names[1], ls='--', lw=1, alpha=0.8, color='darkgray', label='5% FAP', zorder=-20)
    if i==0:
        axes[i].axhline(names[2], ls='dashed', lw=1, alpha=0.8, color='dimgray', label='1% FAP', zorder=-20)
    else:
        axes[i].axhline(names[2], ls='dashed', lw=1, alpha=0.8, color='dimgray', label='', zorder=-20)

axes[0].set_xlim(0.6, 1000)
axes[0].set_ylim(0, 1.1)
axes[0].set_xscale('log')
axes[1].set_xscale('log')
axes[2].set_xscale('log')
axes[3].set_xscale('log')

axes[0].legend(loc='upper right', title='RVs', frameon=False, borderaxespad=0, prop=dict(family='serif', weight='normal', size=9), title_fontsize=12)
axes[1].legend(loc='upper right', title='Residuals', frameon=False, borderaxespad=0, prop=dict(family='serif', weight='normal', size=9), title_fontsize=12)
axes[2].legend(loc='upper right', title='BIS', frameon=False, borderaxespad=0, prop=dict(family='serif', weight='normal', size=9), title_fontsize=12)
axes[3].legend(loc='upper right', title='Window function', frameon=False, borderaxespad=0, prop=dict(family='serif', weight='normal', size=9), title_fontsize=12)
axes[0].tick_params(axis='both', which='major', size=4, top=True, right=True, direction='inout', labelsize=13)
axes[1].tick_params(axis='both', which='major', size=4, top=True, right=True, direction='inout', labelsize=13)
axes[2].tick_params(axis='both', which='major', size=4, top=True, right=True, direction='inout', labelsize=13)
axes[3].tick_params(axis='both', which='major', size=4, top=True, right=True, direction='inout', labelsize=13)
axes[0].tick_params(axis='both', which='minor', size=2, top=True, right=True, direction='inout', labelsize=13)
axes[1].tick_params(axis='both', which='minor', size=2, top=True, right=True, direction='inout', labelsize=13)
axes[2].tick_params(axis='both', which='minor', size=2, top=True, right=True, direction='inout', labelsize=13)
axes[3].tick_params(axis='both', which='minor', size=2, top=True, right=True, direction='inout', labelsize=13)
axes[0].set_yticks([0.2, 0.6, 1.0])
# add a big axis, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axis
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Period (d)", labelpad=-2, fontsize=14)
plt.ylabel("Power", labelpad=-1, fontsize=14)
axes[0].text(0.57, 0.83, 'TOI-1199 b', transform=axes[0].transAxes, fontsize=12, ha='right', fontweight='semibold', fontfamily='serif',
             bbox=dict(facecolor='white', edgecolor='gray', boxstyle='round', pad=0.1, lw=0, alpha=0.8))

formatter = ScalarFormatter()
axes[3].xaxis.set_major_formatter(formatter)
#plt.savefig('periodograms_1199.png', dpi=400, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
# esto lo hice para hacer la tabla de rvs en el latex
# tabla = pd.read_table('./data/rvs/1199_final_rvs.dat', sep='\s+', header=0, names=['time', 'rv', 'rv_err', 'bis'])
# tabla2 = pd.read_table('./data/rvs/1273_final_rvs.dat', sep='\s+', header=0, names=['time', 'rv', 'rv_err', 'bis'])
# for i in range(4):
#     if i < 3:
#         tabla.insert(2*i+1, '&'+str(i), '&')
#         tabla2.insert(2*i+1, '&'+str(i), '&')
#     else:
#         tabla.insert(2*i+1, '&'+str(i), '\\\\')
#         tabla2.insert(2*i+1, '&'+str(i), '\\\\')

# tabla.to_csv('./data/rvs/latex_1199.txt', header=None, index=None, sep=' ', mode='w')
# tabla2.to_csv('./data/rvs/latex_1273.txt', header=None, index=None, sep=' ', mode='w')

## Fit figure paper

In [ ]:
def format_axes(fig):
    for i, ax in enumerate(fig.axes):
        ax.tick_params(axis='both', which='major', size=3, left=True, right=True, top=True, bottom=True, direction='in', labelsize=11)
        ax.tick_params(axis='both', which='minor', size=1.5, left=True, right=True, top=True, bottom=True, direction='in', labelsize=11)
# format_axes(fig)


In [ ]:
fig = plt.figure(figsize=(14, 6), constrained_layout=False)
outer_grid = fig.add_gridspec(4, 4, wspace=0.5, hspace=0.5)

inner_grid1 = outer_grid[0:2, 0:2].subgridspec(1, 2, wspace=0, hspace=0.1)
axs1 = inner_grid1.subplots(sharey=True)

inner_grid2 = outer_grid[2:4, 0:2].subgridspec(1, 2, wspace=0, hspace=0.1)
axs2 = inner_grid2.subplots(sharey=True)

inner_grid3 = outer_grid[0:4, 2:4].subgridspec(2, 1, wspace=0.1, hspace=0, height_ratios=[3, 1])
axs3 = inner_grid3.subplots(sharex=True)

#tess 2 min
x1, y1, yerr1 = datasets['tess_2min'][:3]
x1_fold = (x1 - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# axs1[0].plot(x1_fold*24, y1-tess_2min_mean, marker='o', ls='none', color='lightgrey', 
#             alpha=1, label="TESS 2min", ms=1, zorder=3)
# axs1[0].errorbar(x1_fold*24, y1-tess_2min_mean, yerr=yerr1, fmt='none', color='lightgrey',
#              alpha=0.4, elinewidth=0.5, ecolor='darkgrey', capsize=0, zorder=2)
axs1[0].errorbar(x1_fold*24, y1-tess_2min_mean, yerr=yerr1, fmt="o", color='lightgrey', 
            mec='darkgrey', mew=0.2, ecolor='darkgrey', elinewidth=0.2, label="TESS 2-min", alpha=1, zorder=1, ms=4) 

lcc1 = lk.LightCurve(time=x1_fold, flux=y1-tess_2min_mean, flux_err=yerr1)
lcc1_binned = lcc1.bin(time_bin_size=0.01)
axs1[0].scatter(lcc1_binned['time'].value*24, lcc1_binned['flux'].value, marker='o', s=15,  
                c='k', alpha=0.9, zorder=12)
pred1 = np.percentile(flat_samps["tess_2min_lc_pred"], [16, 50, 84], axis=-1)
axs1[0].plot(phase_lc*24, pred1[1], color='gold', label="model", lw=1.5, zorder=4)
art = axs1[0].fill_between(phase_lc*24, pred1[0], pred1[2], color='gold', alpha=0.5, zorder=5)
art.set_edgecolor("none")
axs1[0].set_xlim(-0.08*24, 0.08*24)
axs1[0].set_ylim(-9, 6)
axs1[0].set_yticks([-8,-4,0,4])
axs1[0].set_ylabel(u'$\Delta$ Flux (‰)', fontsize=12, labelpad=0.5)
axs1[0].set_xlabel("Time (h)", fontsize=12, labelpad=0.5)
axs1[0].text(0.362, 0.9, 'TESS 2-min', transform=axs1[0].transAxes, fontsize=10, ha='right', fontweight='normal', fontfamily='sans-serif',
             bbox=dict(facecolor='white', edgecolor='k', boxstyle='round', pad=0.2, lw=0.5, alpha=0.8))

#tess 30 min
x2, y2, yerr2 = datasets['tess_30min'][:3]
x2_fold = (x2 - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# axs1[1].plot(x2_fold*24, y2-tess_30min_mean, marker='o', ls='none', color='lightgrey',
#             mec='lightgrey', mew=1, alpha=1, label="TESS 30min", ms=4)
# axs1[1].errorbar(x2_fold*24, y2-tess_30min_mean, yerr=yerr2, fmt='none', color='lightgrey',
#              alpha=0.4, elinewidth=0.5, ecolor='darkgrey', capsize=0, zorder=2)

axs1[1].errorbar(x2_fold*24, y2-tess_30min_mean, yerr=yerr2, fmt="o", color='lightgrey', 
            mec='darkgrey', mew=0.2, ecolor='darkgrey', elinewidth=0.2, label="TESS 30-min", alpha=1, zorder=1, ms=4) 

lcc2 = lk.LightCurve(time=x2_fold, flux=y2-tess_30min_mean, flux_err=yerr2)
lcc2_binned = lcc2.bin(time_bin_size=0.02)
axs1[1].scatter(lcc2_binned['time'].value*24, lcc2_binned['flux'].value, marker='o', s=15,
                c='k', alpha=0.9, zorder=12)
pred2 = np.percentile(flat_samps["tess_30min_lc_pred"], [16, 50, 84], axis=-1)
axs1[1].plot(phase_lc*24, pred2[1], color='gold', label="model", lw=1.5)
art = axs1[1].fill_between(phase_lc*24, pred2[0], pred2[2], color='gold', alpha=0.5, zorder=2)
art.set_edgecolor("none")
axs1[1].set_xlim(-0.08*24, 0.08*24)
axs1[1].set_ylim(-9, 6)
axs1[1].set_xlabel("Time (h)", fontsize=12, labelpad=0.5)
axs1[1].text(0.394, 0.9, 'TESS 30-min', transform=axs1[1].transAxes, fontsize=10, ha='right', fontweight='normal', fontfamily='sans-serif',
             bbox=dict(facecolor='white', edgecolor='k', boxstyle='round', pad=0.2, lw=0.5, alpha=0.8))

# keplercam_B
x3, y3, yerr3 = datasets['keplercam_B'][:3]
x3_fold = (x3 - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# axs2[0].plot(x3_fold*24, y3-keplercam_B_mean, marker='o', ls='none', color='lightgrey',
#             mec='lightgrey', mew=1, alpha=1, label="Keplercam B", ms=4)
# axs2[0].errorbar(x3_fold*24, y3-keplercam_B_mean, yerr=yerr3, fmt='o', color=colormap1[2],
#              alpha=0.8, ms=4, zorder=8, elinewidth=0.5, capsize=0)
axs2[0].errorbar(x3_fold*24, y3-keplercam_B_mean, yerr=yerr3, fmt="o", color='lightgrey', 
            mec='darkgrey', mew=0.2, ecolor='darkgrey', elinewidth=0.2, label="Keplercam B", alpha=1, zorder=1, ms=4) 

lcc3 = lk.LightCurve(time=x3_fold, flux=y3-keplercam_B_mean, flux_err=yerr3)
lcc3_binned = lcc3.bin(time_bin_size=0.01)
axs2[0].scatter(lcc3_binned['time'].value*24, lcc3_binned['flux'].value, marker='o', s=15,
                c='k', alpha=0.9, zorder=12)
pred3 = np.percentile(flat_samps["keplercam_B_lc_pred"], [16, 50, 84], axis=-1)
axs2[0].plot(phase_lc*24, pred3[1], color=colormap1[2], label="model", lw=1.5)
art = axs2[0].fill_between(phase_lc*24, pred3[0], pred3[2], color=colormap1[2], alpha=0.5, zorder=2)
art.set_edgecolor("none")
axs2[0].set_xlim(-0.08*24, 0.08*24)
axs2[0].set_ylim(-9, 6)
axs2[0].set_yticks([-8,-4,0,4])
axs2[0].set_ylabel(u'$\Delta$ Flux (‰)', fontsize=12, labelpad=0.5)
axs2[0].set_xlabel("Time (h)", fontsize=12, labelpad=0.5)
axs2[0].text(0.3915, 0.9, 'Keplercam B', transform=axs2[0].transAxes, fontsize=10, ha='right', fontweight='normal', fontfamily='sans-serif',
             bbox=dict(facecolor='white', edgecolor='k', boxstyle='round', pad=0.2, lw=0.5, alpha=0.8))

# keplercam_z
x4, y4, yerr4 = datasets['keplercam_z'][:3]
x4_fold = (x4 - t0_ + 0.5 * p_) % p_ - 0.5 * p_
# axs2[1].plot(x4_fold*24, y4-keplercam_z_mean, marker='o', ls='none', color='lightgrey',
#             mec='lightgrey', mew=1, alpha=1, label="Keplercam z", ms=4)
# axs2[1].errorbar(x4_fold*24, y4-keplercam_z_mean, yerr=yerr4, fmt='o', color=colormap1[3],
#              alpha=0.8, ms=4, zorder=8, elinewidth=0.5, capsize=0)
axs2[1].errorbar(x4_fold*24, y4-keplercam_z_mean, yerr=yerr4, fmt="o", color='lightgrey', 
            mec='darkgrey', mew=0.2, ecolor='darkgrey', elinewidth=0.2, label="Keplercam z", alpha=1, zorder=1, ms=4) 
lcc4 = lk.LightCurve(time=x4_fold, flux=y4-keplercam_z_mean, flux_err=yerr4)
lcc4_binned = lcc4.bin(time_bin_size=0.01)
axs2[1].scatter(lcc4_binned['time'].value*24, lcc4_binned['flux'].value, marker='o', s=15,
                c='k', alpha=0.9, zorder=12)
pred4 = np.percentile(flat_samps["keplercam_z_lc_pred"], [16, 50, 84], axis=-1)
axs2[1].plot(phase_lc*24, pred4[1], color=colormap1[6], label="model", lw=1.5)
art = axs2[1].fill_between(phase_lc*24, pred4[0], pred4[2], color=colormap1[6], alpha=0.5, zorder=2)
art.set_edgecolor("none")
axs2[1].set_xlim(-0.08*24, 0.08*24)
axs2[1].set_ylim(-9, 6)
axs2[1].set_xlabel("Time (h)", fontsize=12, labelpad=0.5)
axs2[1].text(0.385, 0.9, 'Keplercam z', transform=axs2[1].transAxes, fontsize=10, ha='right', fontdict=dict(fontweight='normal', fontfamily='sans-serif'),
            bbox=dict(facecolor='white', edgecolor='k', boxstyle='round', pad=0.2, lw=0.5, alpha=0.8))


# Plot RVs
# Plot the folded data
x_fold = (x_rv - t0_ + 0.5 * p_) % p_ - 0.5 * p_
axs3[0].errorbar(x_fold/p_, y_rv - bkg_, yerr=np.sqrt(yerr_rv**2+rv_jitter**2), fmt="o", color='lightgrey', 
            markeredgecolor='darkgrey', ecolor='darkgrey', elinewidth=0.5, label='SOPHIE RV\'s', alpha=1, zorder=1) 

# Compute the posterior prediction for the folded RV model 
t_fold = (t_rv - t0_ + 0.5 * p_) % p_ - 0.5 * p_
inds = np.argsort(t_fold)
pred = np.percentile(trace.posterior["vrad_pred"].values[:, :, inds],[16, 50, 84],axis=(0, 1),)
axs3[0].plot(t_fold[inds]/p_, pred[1], color="#FF7F00", label="Model", zorder=2, lw=1.5)
art = axs3[0].fill_between(t_fold[inds]/p_, pred[0], pred[2], color="#FF7F00", alpha=0.5, zorder=2) #color="#988ED5"
art.set_edgecolor("none")
axs3[0].set_xlim(-0.51, 0.51)
axs3[0].set_ylim(-58, 70)
axs3[0].set_ylabel("RV (m s$^{-1}$)", labelpad=1, fontsize=12)
axs3[0].text(0.195, 0.935, 'SOPHIE RV\'s', transform=axs3[0].transAxes, fontsize=10, ha='right', fontweight='normal', 
             fontfamily='sans-serif', bbox=dict(facecolor='white', edgecolor='k', boxstyle='round', pad=0.2, lw=0.5, alpha=0.8))

# Plot the folded residuals
axs3[1].axhline(y=0, ls='--', color='k', linewidth=1, alpha=0.5, zorder=1)
axs3[1].errorbar(x_fold/p_, y_rv-rv_model_, yerr=np.sqrt(yerr_rv**2+rv_jitter**2), fmt="o", color='lightgray',
            markeredgecolor='darkgray', ecolor='darkgray', elinewidth=0.5, alpha=1, label='residuals', zorder=2)
axs3[1].set_ylabel('Residuals', labelpad=1, fontsize=12)
axs3[1].set_xlabel("Phase", labelpad=1, fontsize=12)
axs3[1].set_xlim(-0.51, 0.51)
axs3[1].set_ylim(-40, 45)
axs3[1].set_xticks([-0.5, 0, 0.5])
axs3[1].set_yticks([-30, 0, 30])

format_axes(fig)
#plt.savefig('1199_fits.png', dpi=300, bbox_inches='tight')

# TTVs?

In [ ]:
#t0_, p_
print(xlong_)
print(t0_)

In [ ]:
transit_times = xo.orbits.ttv.compute_expected_transit_times(-844, -100, p_, t0_)

In [ ]:
plt.plot(xlong_, ylong_, 'k.', ms=1)
# plt.xlim(-850,-840)

In [ ]:
# un punto a la izquierda de cada transito, son 10
pcentros = [-633, -636.5, -640, -647.5, -651, -655, -823.5, -827.5, -838.5, -842 ]

In [ ]:
# usemos uncertainties para propagar el error a Tc
t0_error = ufloat(t0_, t0_err)
p_error = ufloat(p_, p_err)

In [ ]:
# define a list of 10 diferent colors
colorcycle = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd',
              '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))
for n, i in enumerate(pcentros):
    npe = np.floor((i-t0_)/p_)+1
    tc = t0_error + p_error*(npe)
    xlong_2 = xlong_[np.abs(xlong_ - tc.nominal_value) < 0.08*p_]
    ylong_2 = ylong_[np.abs(xlong_ - tc.nominal_value) < 0.08*p_]
    yerrlong_2 = yerrlong_[np.abs(xlong_ + tc.nominal_value) < 0.08*p_]
    xlong_2_err = []
    for k in xlong_2:
        #xlong_2_err.append((k - tc)/p_error)
        xlong_2_err.append((k - npe*p_error-t0_))
    #ax.set_color_cycle([plt.cm.spectral(i) for i in np.linspace(0, 1, 10)])
    for r, m in enumerate(xlong_2_err):
        plt.errorbar(x=m.nominal_value*24, y=ylong_2[r], xerr=m.s*24, fmt='o', color=colorcycle[n], ms=3)
plt.axvline(0, color="k", ls='--', lw=1, alpha=0.7)
plt.xlim(-0.08*24, 0.08*24)
#plt.savefig('1199_ttvs.png', dpi=300, bbox_inches='tight')

# RV trend results

In [ ]:
bkg_trend = np.percentile(trace.posterior["bkg"].values[:, :,],[16, 50, 84], axis=(0, 1))
bkg_trend2 = np.percentile(trace.posterior["bkg"].values[:, :,],[0.15, 50, 99.85], axis=(0, 1))
fig, (ax1, ax2) = plt.subplots(2, figsize=(8, 6), sharex=True)
fig.subplots_adjust(hspace=0)

ax1.plot(x_rv, bkg_trend[1], color="k", label="Background model TOI-1199")
art = ax1.fill_between(x_rv, bkg_trend[0], bkg_trend[2], color="#988ED5", alpha=0.5, label='1-sigma') 
art.set_edgecolor("none")
ax1.legend(loc='upper right', fontsize=10, frameon=True, markerscale=1, handlelength=1, handletextpad=0.5)
ax1.set_ylabel("RV (m s$^{-1}$)", labelpad=1, fontsize=12)
dataset_length = x_rv.max() - x_rv.min()
max_k = bkg_trend[2].max() - bkg_trend[0].min()
# write the max drift in the plot
ax1.text(0.05, 0.9, 'Max drift: {:.2f} m/s in {:.2f} days'.format(max_k, dataset_length), transform=ax1.transAxes, fontsize=12,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
# tick params
ax1.tick_params(axis='both', which='major', labelsize=12, direction='in', top=True, right=True, bottom=True, left=True)
ax1.tick_params(axis='both', which='minor', labelsize=12, direction='in', top=True, right=True, bottom=True, left=True)
ax2.plot(x_rv, bkg_trend2[1], color="k", label="Background model TOI-1199")
art = ax2.fill_between(x_rv, bkg_trend2[0], bkg_trend2[2], color="#988ED5", alpha=0.5, label='3-sigma')
art.set_edgecolor("none")
ax2.legend(loc='upper right', fontsize=10, frameon=True, markerscale=1, handlelength=1, handletextpad=0.5)
ax2.set_xlabel("Time [BJD - 2450000]", labelpad=1, fontsize=12)
ax2.set_ylabel("RV (m s$^{-1}$)", labelpad=1, fontsize=12)
max_k2 = bkg_trend2[2].max() - bkg_trend2[0].min()
# write the max drift in the plot
ax2.text(0.05, 0.9, 'Max drift: {:.2f} m/s'.format(max_k2), transform=ax2.transAxes, fontsize=12,
        verticalalignment='top', bbox=dict(boxstyle='round', facecolor='white', alpha=0.5))
# tick params
ax2.tick_params(axis='both', which='major', labelsize=12, direction='in', top=True, right=True, bottom=True, left=True)
ax2.tick_params(axis='both', which='minor', labelsize=12, direction='in', top=True, right=True, bottom=True, left=True)
#plt.savefig('drift_1199.png', dpi=300, bbox_inches='tight', facecolor='white')
plt.show()

In [ ]:
mass_max = max_k*np.sqrt(1-eccen.s**2)*(periodo*u.day.to(u.second)*(trace.posterior["m_star"].median().item()*u.M_sun.to(u.kg))**2/(2*np.pi*constants.G))**(1/3)*u.kg.to(u.M_jup)
mass_max

In [ ]:
892.71*2

In [ ]:
# pred = np.percentile(flat_samps["bkg_lc_pred"], [16, 50, 84], axis=-1)

# Additional LCs

CROW

In [ ]:
#crow = pd.read_table('./data/photom/TOI1199/CROW/crow_measurements.dat')
crow = pd.read_table('./data/photom/TOI1199/CROW/TIC_99869022_01_20200204_CROW_g_measurements.tbl', sep='\s+')
# names of the columns are: 'rel_flux_T1_dn', 'rel_flux_err_T1_dn', 'BJD_TDB'

In [ ]:
# crow['flux'] = ((crow['rel_flux_T1']/(crow['rel_flux_T1'].median())))*1e3
# crow['flux_err'] = ((crow['rel_flux_err_T1']/(crow['rel_flux_T1'].median())))*1e3
# crow['time'] = crow['BJD_TDB']-2457000-ref_time

crow['time'] = crow['BJD_TDB']-2457000-ref_time
crow['flux'] = crow['rel_flux_T1_dfn']
crow['flux_err'] = crow['rel_flux_err_T1_dfn']


In [ ]:
#puntos?
print('puntos:', len(crow))
# texp?
print('texp:', np.median(np.diff(crow['BJD_TDB']))*24*60*60)

In [ ]:
plt.subplots(figsize=(12,5))
plt.plot(crow['time'], (crow['flux']*1e3-1e3), 'o', label='CROW')
#plt.plot(crow['time'], crow2['rel_flux_T1_dn'], 'o', label='CROW')
#plt.ylim(-20,20)
plt.show()

In [ ]:
from scipy.stats import binned_statistic
asd = binned_statistic(crow['time'], crow['rel_flux_T1_dn']*1e3-1e3, bins=len(crow)/2)
asd_time = binned_statistic(crow['time'], crow['time'], bins=len(crow)/2)
len(asd_time[0])

In [ ]:
plt.subplots(figsize=(12,5))
plt.plot(asd_time[0], asd[0], 'o', label='cat')
plt.ylim(-20,20)
plt.show()

WBRO

In [ ]:
wbro = pd.read_table('./data/photom/TOI1199/WBRO/TIC99869022-01_20200204_WBRO-24cm_R_measurements.tbl')

In [ ]:
# wbro['flux'] = ((wbro['rel_flux_T1']/(wbro['rel_flux_T1'].median())))*1e3
# wbro['flux_err'] = ((wbro['rel_flux_err_T1']/(wbro['rel_flux_T1'].median())))*1e3
wbro['time'] = wbro['BJD_TDB']-2457000-ref_time

wbro['flux'] = wbro['rel_flux_T1_dfn']
wbro['flux_err'] = wbro['rel_flux_err_T1_dfn']

In [ ]:
#puntos?
print('puntos:', len(wbro))
# texp?
print('texp:', np.median(np.diff(wbro['BJD_TDB']))*24*60*60)

In [ ]:
plt.subplots(figsize=(12,5))
plt.plot(wbro['time'], wbro['flux']*1e3-1e3, 'o', label='wbro')
plt.show()

WCO

In [ ]:
wco = pd.read_table('./data/photom/TOI1199/WCO/TIC99869022-01_20201209_WCO_gp_measurements.tbl')

In [ ]:
# wco['flux'] = ((wco['rel_flux_T1']/(wco['rel_flux_T1'].median())))
# wco['flux_err'] = ((wco['rel_flux_err_T1']/(wco['rel_flux_T1'].median())))
wco['time'] = wco['BJD_TDB']-2457000-ref_time
wco['flux'] = wco['rel_flux_T1_dfn']
wco['flux_err'] = wco['rel_flux_err_T1_dfn']

In [ ]:
print('puntos:', len(wco))
print('texp:', np.median(np.diff(wco['BJD_TDB']))*24*60*60)

In [ ]:
plt.subplots(figsize=(12,5))
plt.plot(wco['time'], wco['flux']*1e3-1e3, 'o', label='wco')
# horizontal line at 0
plt.axhline(0, color='k', lw=1)
plt.show()

## Extra LCs Phase plot

In [ ]:
extra_datasets = OrderedDict(
    [
        ('muscat_g', [x_muscat_g, y_muscat_g, yerr_muscat_g]),
        ('muscat_r', [x_muscat_r, y_muscat_r, yerr_muscat_r]),
        ('muscat_i', [x_muscat_i, y_muscat_i, yerr_muscat_i]),
        ('muscat_zs', [x_muscat_zs, y_muscat_zs, yerr_muscat_zs]),
        ("CROW", [crow['time'], crow['flux']*1e3-1e3, crow['flux_err']*1e3]),
        ("WCO", [wco['time'], wco['flux']*1e3-1e3, wco['flux_err']*1e3]),
        ("WBRO", [wbro['time'], wbro['flux']*1e3-1e3, wbro['flux_err']*1e3])
    ]
)
# crow es g'
# wbro es Rc
# wco es g'

In [ ]:
extra_datasets.keys()

In [ ]:
fig, axes = plt.subplots(7, sharex=True, sharey=True, figsize=(5, 14))
plt.subplots_adjust(hspace=0.0)
# add a big axis, hide frame
fig.add_subplot(111, frameon=False)
# hide tick and tick label of the big axi
plt.tick_params(labelcolor='none', which='both', top=False, bottom=False, left=False, right=False)
plt.xlabel("Time (h)", fontsize=11, labelpad=-5)
plt.title('TOI-1199', fontsize=11)
for n, name in enumerate(extra_datasets):
    ax = axes[n]
    # plot the folded transits
    x, y, yerr = extra_datasets[name][:3]
    folded = (x - t0_ + 0.5 * p_) % p_ - 0.5 * p_
    mean = np.median(flat_samps["tess_2min_mean"])
    pred = np.percentile(flat_samps["tess_2min_lc_pred"], [16, 50, 84], axis=-1)
    if n==0:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="MusCAT2 g'", ms=4)
    elif n==1:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="MusCAT2 r'", ms=4)
    elif n==2:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="MusCAT2 i'", ms=4)
    elif n==3:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="MusCAT2 z$_s$'", ms=4)
    elif n==4:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="CROW g'", ms=4)
    elif n==5:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="WCO Rc", ms=4)
    else:
        ax.plot(folded*24, (y  - mean), marker='o', ls='none', color='lightgray', 
            mec='darkgray', mew=0.5, alpha=1, label="WBRO g'", ms=4)
    ax.errorbar(folded*24, (y  - mean), yerr=yerr, color='darkgray', ls='none', 
                alpha=0.2, elinewidth=0.5, capsize=0, zorder=-10)
    ax.plot(phase_lc*24, pred[1], color='k', label="Best fit model (TESS)", 
            linewidth=1.5, alpha=1)
    art = ax.fill_between(phase_lc*24, pred[0], pred[2], color='k', 
                            alpha=0.65)
    art.set_edgecolor("none")
    ax.set_ylabel(u'$\Delta Flux$ (‰)', fontsize=11, labelpad=-3)
    ax.legend(fontsize=8, loc=2, handlelength=0.5, handletextpad=0.5, borderpad=0.4, markerscale=0.8, 
                framealpha=1, frameon=True)
    ax.tick_params(axis='y', which='major', size=2, left=True, right=True, direction='in', labelsize=11)
    ax.tick_params(axis='y', which='minor', size=2, left=True, right=True, direction='in', labelsize=11)
    ax.tick_params(axis='x', which='major', size=2, bottom=True, top=True, direction='in', labelsize=11)

axes[-1].set_xlim(-0.08*24, 0.08*24)
axes[-1].set_ylim(-16, 12)
axes[-1].set_xticks([-1,0,1])
#plt.savefig('1199_extralcs.png', dpi=1200, bbox_inches='tight', facecolor='white')
plt.show()